<div style="font-size: 48px;">Towards a JSON Schema for the Language Server Protocol</div>

_Thoughts on a [machine-readable][], language-agnostic specification of the
[Language Server Protocol][] in [JSON Schema][]._

**tl,dr: Using [ts-json-schema-generator][] with some code generated from the spec yields a fairly robust schema ([JSON](./generated.protocol.schema.json) or [YAML](./generated.protocol.schema.yml)).**

[machine-readable]: https://github.com/microsoft/language-server-protocol/issues/67
[Language Server Protocol]: https://microsoft.github.io/language-server-protocol
[JSON Schema]: https://json-schema.org
[ts-json-schema-generator]: https://github.com/vega/ts-json-schema-generator

> Viewable on:
> - [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gist/bollwyvl/7a128978b8ae89ab02bbd5b84d07a4b7/master?urlpath=lab%2Ftree%2FTowards%20a%20JSON%20Schema%20for%20the%20Language%20Server%20Protocol.ipynb)
> - [nbviewer](https://nbviewer.jupyter.org/gist/bollwyvl/7a128978b8ae89ab02bbd5b84d07a4b7/Towards%20a%20JSON%20Schema%20for%20the%20Language%20Server%20Protocol.ipynb)
> - [gist](https://gist.github.com/bollwyvl/7a128978b8ae89ab02bbd5b84d07a4b7)

# Motivation

The Language Server community would benefit from a more [machine-readable](https://github.com/microsoft/language-server-protocol/issues/67) description of the Language Server Protocol Specification.

JSON Schema, with [implementations](https://json-schema.org/implementations.html) across a number of languages is a fine initial choice, and would ease the way toward other specifications.

# What is a useful level of schema?

For the purposes of:
- automated testing
- language-indpendent validation

A few top-level artifacts would be useful:
- standalone messages
- message sequences (e.g. `[request, response, ...]`)

# What is "canonical"?
The Language Server Protocol specification is basically split between:
- the canonical [spec](https://github.com/microsoft/language-server-protocol/blob/gh-pages/_specifications/specification-3-14.md)
    - markdown
    - apparently has no automated verification
    - changes its structure over time
- the [reference implementation](https://github.com/microsoft/vscode-languageserver-node/blob/master/protocol/src/protocol.ts)
    - typescript
    - has robust automated testing
    - widely adopted, even for non js/ts-adjacent languages

## On versions
The two repos are released at different cadences, as are the tag and branch conventions. For this exploation, we'll look at two commits, basically what the author was able to pull down on `2019-10-19`.

In [1]:
LSP_COMMIT = "639528c"
VLSPN_COMMIT = "6a7e832"

In [2]:
import json, jsonschema, pathlib, yaml, IPython, re, pandas, pyemojify, jinja2, copy, pytest

### Getting the spec

In [3]:
if not pathlib.Path("language-server-protocol").exists():
    !git clone https://github.com/microsoft/language-server-protocol.git
!cd language-server-protocol && git checkout -f {LSP_COMMIT} && git describe HEAD

if not pathlib.Path("vscode-languageserver-node").exists():
    !git clone https://github.com/microsoft/vscode-languageserver-node.git
!cd vscode-languageserver-node && git checkout -f {VLSPN_COMMIT} && git describe HEAD

HEAD is now at 639528c Another attempt to fix the redirect.
release/3.13.0-210-g639528c
HEAD is now at 6a7e832 Clarify insertTextFormat default value
release/client/6.0.0-next.1-2-g6a7e832


# Parsing the Spec

## Parsing the Markdown spec
The canonical document is represented in Markdown. It's _standardized enough_ to be parsed with text, though a more thorough parsing might prove required. At present, a simple version-based naming scheme is used for the current spec (`3.14`) and the next spec (`3,1`). Both change.

In [4]:
spec_md = pathlib.Path("language-server-protocol") / "_specifications" / "specification-3-14.md"
spec = spec_md.read_text()

In [5]:
md_features = spec.split("#### $ Notifications and Requests")[1].split("### Implementation considerations")[0].split("#### <a href")[1:]
print(len(md_features), "features defined")

50 features defined


## Parsing the reference implementation
There are several, separately developed [implementations](https://github.com/vega/ts-json-schema-generator/issues/101) of TS-to-JSON-Schema, each with their own quirks. The most robust (and maintained) appears to be [ts-json-schema-generator](https://github.com/vega/ts-json-schema-generator).

### The Generator
The most robust transformer is [vega/ts-json-schema-generator](https://github.com/vega/ts-json-schema-generator). We'll install it (from a fork where we'll add needed features based on the VLSPN code base) into the VLSPN environment, as well as all the dependencies.

In [6]:
if not pathlib.Path("ts-json-schema-generator").exists():
    !git clone https://github.com/bollwyvl/ts-json-schema-generator
!cd ts-json-schema-generator && git checkout -f master && git reset --hard origin/master
!cd ts-json-schema-generator && jlpm && jlpm build
!cd vscode-languageserver-node && npm install --save --only=dev ../ts-json-schema-generator prettier && npm install

Already on 'master'
Your branch is up to date with 'origin/master'.
HEAD is now at 5efb7a4 whitespace
yarn install v1.15.2
[1/5] Validating package.json...
[2/5] Resolving packages...
⠁ success Already up-to-date.
Done in 0.43s.
yarn run v1.15.2
$ tsc
Done in 2.65s.
audited 334 packages in 4.137s/ postinstall: sill install executeActions0m.18.K
found 0 vulnerabilities

   ...........] - postinstall: sill install executeActions0mlTreK
> vscode-lsp@1.0.0 postinstall /home/weg/Documents/projects/lsp-json-schema/vscode-languageserver-node
> cd types && npm install && cd ../jsonrpc && npm install && cd ../protocol && npm install && cd ../server && npm install && cd ../client && npm install && cd ../client-tests && npm install && cd ..

up to date in 0.417s....] \ prepare:types: info lifecycle vscode-languageserver-type
found 0 vulnerabilities

up to date in 0.478s....] - prepare:jsonrpc: info lifecycle vscode-jsonrpc@5.0.0-nex
found 0 vulnerabilities

audited 2 packages in 0.761s] \ prepare

### Generating the schema

In [7]:
!rm naive.protocol.schema.json
!cd vscode-languageserver-node/protocol && ../node_modules/.bin/ts-json-schema-generator -c --path './src/protocol.ts' --expose all > ../../naive.protocol.schema.json && wc ../../naive.protocol.schema.json

  3518  11230 125799 ../../naive.protocol.schema.json


In [8]:
naive_schema = json.loads(pathlib.Path("naive.protocol.schema.json").read_text())
IPython.display.JSON(naive_schema)

<IPython.core.display.JSON object>

> If you are reading this statically, don't worry, there will be schema to read later...

# Exploring the Spec

To validate the robustness of combining the two parsed structures, we'll transform the raw markdown into a tabular format. First we break up the markdown with regexen.

In [9]:
df = pandas.DataFrame(md_features, columns=["_md"])
md = df["_md"]
df["method"] = md.apply(lambda md: re.findall(r"""\* method: '(.*)'""", md)[0])
df["_raw_params"] = md.apply(lambda md: re.findall(r"""\* params: (.*)""", md)[0])
df["title"] = md.apply(lambda md: md.split(">")[1].split("<")[0].strip().split("(")[0].strip())
df["type"] = md.apply(lambda md: pyemojify.emojify(md.split(">")[1].split("<")[0].strip().split("(")[1]).replace(")","").strip())
df["_raw_result"] = md.apply(lambda md: (re.findall(r"""\* result: (.*)""", md, flags=re.M | re.I) or [None])[0])
df["_raw_error"] = md.apply(lambda md: (re.findall(r"""\* error: (.*)""", md, flags=re.M | re.I) or [None])[0])
df = df.sort_values(["type", "method"]).set_index(["type", "method"])
df[set(df.columns) - set(["_md"])].T

type                                                        ↩️  \
method                                        codeLens/resolve   
_raw_error   code and message set in case an exception happ...   
title                                Code Lens Resolve Request   
_raw_params                                         `CodeLens`   
_raw_result                                         `CodeLens`   

type                                                            \
method                                  completionItem/resolve   
_raw_error   code and message set in case an exception happ...   
title                          Completion Item Resolve Request   
_raw_params                                   `CompletionItem`   
_raw_result                                   `CompletionItem`   

type                                                            \
method                                    documentLink/resolve   
_raw_error   code and message set in case an exception happ...   
title                            Document Link Resolve Request   
_raw_params                                     `DocumentLink`   
_raw_result                                     `DocumentLink`   

type                                                 \
method                                   initialize   
_raw_error                                     None   
title                            Initialize Request   
_raw_params  `InitializeParams` defined as follows:   
_raw_result  `InitializeResult` defined as follows:   

type                                                            \
method                                                shutdown   
_raw_error   code and message set in case an exception happ...   
title                                         Shutdown Request   
_raw_params                                               void   
_raw_result                                               null   

type                                                            \
method                                 textDocument/codeAction   
_raw_error   code and message set in case an exception happ...   
title                                      Code Action Request   
_raw_params             `CodeActionParams` defined as follows:   
_raw_result  `(Command | CodeAction)[]` \| `null` where `Co...   

type                                                            \
method                                   textDocument/codeLens   
_raw_error   code and message set in case an exception happ...   
title                                        Code Lens Request   
_raw_params               `CodeLensParams` defined as follows:   
_raw_result         `CodeLens[]` \| `null` defined as follows:   

type                                                            \
method                          textDocument/colorPresentation   
_raw_error   code and message set in case an exception happ...   
title                               Color Presentation Request   
_raw_params       `ColorPresentationParams` defined as follows   
_raw_result          `ColorPresentation[]` defined as follows:   

type                                                            \
method                                 textDocument/completion   
_raw_error   code and message set in case an exception happ...   
title                                       Completion Request   
_raw_params             `CompletionParams` defined as follows:   
_raw_result  `CompletionItem[]` \| `CompletionList` \| `nul...   

type                                                            ...  \
method                                textDocument/declaration  ...   
_raw_error   code and message set in case an exception happ...  ...   
title                                 Goto Declaration Request  ...   
_raw_params  [`TextDocumentPositionParams`](#textdocumentpo...  ...   
_raw_result  [`Location`](#location) \| [`Location`](#locat...  ...   

type                                                      ➡️  \
me

We'd like some better types structure.

Results are more complicated, typographically, so we'll wait until we have the schema.

## Joining to Schema
Perhaps we can use what we got from the naive schema...

In [10]:
def parse_params(rp, p, schema):
    if p in schema["definitions"]:
        return schema["definitions"][p]
    if rp in ['void', 'none']:
        return {"type": "null"}
    if rp in ["'any'"]:
        return {}
    return None

In [11]:
df["params"] = df["_raw_params"].apply(lambda rp: (re.findall(r'`(.*?)`', rp) or [None])[0])

In [12]:
df["params_schema"] = [
    parse_params(row["_raw_params"], row["params"], naive_schema)
    for row in df[["_raw_params", "params"]].to_dict(orient="records")
]
df[["params_schema", "params"]].T

type                        ↩️                                              \
method        codeLens/resolve completionItem/resolve documentLink/resolve   
params_schema             None                   None                 None   
params                CodeLens         CompletionItem         DocumentLink   

type                                                              \
method                                                initialize   
params_schema  {'additionalProperties': False, 'properties': ...   
params                                          InitializeParams   

type                             \
method                 shutdown   
params_schema  {'type': 'null'}   
params                     None   

type                                                              \
method                                   textDocument/codeAction   
params_schema  {'additionalProperties': False, 'description':...   
params                                          CodeActionParams   

type                                                              \
method                                     textDocument/codeLens   
params_schema  {'additionalProperties': False, 'description':...   
params                                            CodeLensParams   

type                                          \
method        textDocument/colorPresentation   
params_schema                           None   
params               ColorPresentationParams   

type                                                              \
method                                   textDocument/completion   
params_schema  {'additionalProperties': False, 'description':...   
params                                          CompletionParams   

type                                                              ...  \
method                                  textDocument/declaration  ...   
params_schema  {'additionalProperties': False, 'description':...  ...   
params                                TextDocumentPositionParams  ...   

type                                                          ➡️  \
method                                      textDocument/didSave   
params_schema  {'additionalProperties': False, 'description':...   
params                                 DidSaveTextDocumentParams   

type                                                              \
method                                     textDocument/willSave   
params_schema  {'additionalProperties': False, 'description':...   
params                                WillSaveTextDocumentParams   

type                                                              \
method                          workspace/didChangeConfiguration   
params_schema  {'additionalProperties': False, 'description':...   
params                              DidChangeConfigurationParams   

type                                                              \
method                           workspace/didChangeWatchedFiles   
params_schema  {'additionalProperties': False, 'description':...   
params                               DidChangeWatchedFilesParams   

type                                                        ➡️ ⬅️  \
method        workspace/didChangeWorkspaceFolders $/cancelRequest   
params_schema                                None            None   
params            DidChangeWorkspaceFoldersParams    CancelParams   

type                       ⬅️  \
method        telemetry/event   
params_schema              {}   
params                   None   

type                                                              \
method                           textDocument/publishDiagnostics   
params_schema  {'additionalProperties': False, 'description':...   
params                                  PublishDiagnosticsParams   

type                                                              \
method                                         window/logMessage   
params_schema  {'additionalProperties': False, 'descripti

In [13]:
def parse_results(rr):
    if rr in [None]:
        return rr
    r = (
        rr.replace("defined as follows:", "")
        .replace("`", "")
        .replace("the selected", "")
        .replace(r"\|", "|")
        .replace("An array of DocumentLink", "DocumentLink[]")
    )
    r = re.sub(r"\[(.*?)\]\(#.+\)", "\\1", r).strip()
    for splitter in [' where ', ' describing ', ' as defined ', '. ', ' if ']:
        r = r.split(splitter)[0].strip()
    if r.endswith('.'):
        r = r[:-1]
    return r
df["result"] = df["_raw_result"].apply(lambda rr: parse_results(rr))

In [14]:
df[["_raw_result", "result"]]

_raw_result  \
type  method                                                                                   
↩️    codeLens/resolve                                                            `CodeLens`   
      completionItem/resolve                                                `CompletionItem`   
      documentLink/resolve                                                    `DocumentLink`   
      initialize                                      `InitializeResult` defined as follows:   
      shutdown                                                                          null   
      textDocument/codeAction              `(Command | CodeAction)[]` \| `null` where `Co...   
      textDocument/codeLens                       `CodeLens[]` \| `null` defined as follows:   
      textDocument/colorPresentation               `ColorPresentation[]` defined as follows:   
      textDocument/completion              `CompletionItem[]` \| `CompletionList` \| `nul...   
      textDocument/declaration             [`Location`](#location) \| [`Location`](#locat...   
      textDocument/definition              [`Location`](#location) \| [`Location`](#locat...   
      textDocument/documentColor                    `ColorInformation[]` defined as follows:   
      textDocument/documentHighlight       `DocumentHighlight[]` \| `null` defined as fol...   
      textDocument/documentLink                        An array of `DocumentLink` \| `null`.   
      textDocument/documentSymbol          `DocumentSymbol[]` \| `SymbolInformation[]` \|...   
      textDocument/foldingRange                  `FoldingRange[] | null` defined as follows:   
      textDocument/formatting              [`TextEdit[]`](#textedit) \| `null` describing...   
      textDocument/hover                               `Hover` \| `null` defined as follows:   
      textDocument/implementation          [`Location`](#location) \| [`Location`](#locat...   
      textDocument/onTypeFormatting        [`TextEdit[]`](#textedit) \| `null` describing...   
      textDocument/prepareRename           [`Range`](#range) \| `{ range: Range, placehol...   
      textDocument/rangeFormatting         [`TextEdit[]`](#textedit) \| `null` describing...   
      textDocument/references                            [`Location`](#location)[] \| `null`   
      textDocument/rename                  [`WorkspaceEdit`](#workspaceedit) \| `null` de...   
      textDocument/signatureHelp               `SignatureHelp` \| `null` defined as follows:   
      textDocument/typeDefinition          [`Location`](#location) \| [`Location`](#locat...   
      textDocument/willSaveWaitUntil                                                    None   
      workspace/executeCommand                                               `any` \| `null`   
      workspace/symbol                     `SymbolInformation[]` \| `null` as defined above.   
↪️    client/registerCapability                                                        void.   
      client/unregisterCapability                                                      void.   
      window/showMessageRequest            the selected `MessageActionItem` \| `null` if ...   
      workspace/applyEdit                   `ApplyWorkspaceEditResponse` defined as follows:   
      workspace/configuration                                                          any[]   
      workspace/workspaceFolders              `WorkspaceFolder[] | null` defined as follows:   
➡️    exit                                                                              None   
      initialized                                                                       None   
      textDocument/didChange                                                            None   
      textDocument/didClose                                                             None   
      textDocument/didOpen                                                              None   
      textDocument/didSave                       

In [15]:
def result_to_schema(r, schema):
    if not r:
        return r
    opts = []
    if r in ["void", "void."]:
        return {"type": "null"}

    for sr in [sr.strip() for sr in r.split("\|")]:
        is_arr = False
        if sr.endswith("[]"):
            is_arr = True
            sr = sr[:-2]
        if sr.startswith("null"):
            sr = {"type": "null"}
        elif sr.startswith("void"):
            sr = {"type": "null"}
        elif sr in schema["definitions"]:
            sr = {"$ref": f"#/definitions/{sr}"}
            if is_arr:
                sr = {"type": "array", "items": sr}
        opts += [sr]
    if str in [type(sr) for sr in opts]:
        print(r)
        print(f"\t{opts}")
        return None
    return {"oneOf": opts}
df["result_schema"] = df["result"].apply(lambda rr: result_to_schema(rr, naive_schema))
df[["result"]].T

CodeLens
	['CodeLens']
CompletionItem
	['CompletionItem']
DocumentLink
	['DocumentLink']
InitializeResult
	['InitializeResult']
(Command | CodeAction)[] | null
	['(Command | CodeAction)[] | null']
CodeLens[] | null
	['CodeLens[] | null']
ColorPresentation[]
	['ColorPresentation']
CompletionItem[] | CompletionList | null
	['CompletionItem[] | CompletionList | null']
Location[] |null
	['Location[] |null']
Location[] | null
	['Location[] | null']
ColorInformation[]
	['ColorInformation']
DocumentHighlight[] | null
	['DocumentHighlight[] | null']
DocumentLink[] | null
	['DocumentLink[] | null']
DocumentSymbol[] | SymbolInformation[] | null
	['DocumentSymbol[] | SymbolInformation[] | null']
FoldingRange[] | null
	['FoldingRange[] | null']
TextEdit[] | null
	['TextEdit[] | null']
Hover | null
	['Hover | null']
Location[] | null
	['Location[] | null']
TextEdit[] | null
	['TextEdit[] | null']
Range | { range: Range, placeholder: string } | null
	['Range | { range: Range, placeholder: string } |

type                 ↩️                                              \
method codeLens/resolve completionItem/resolve documentLink/resolve   
result         CodeLens         CompletionItem         DocumentLink   

type                                                                \
method        initialize shutdown          textDocument/codeAction   
result  InitializeResult     null  (Command | CodeAction)[] | null   

type                                                         \
method textDocument/codeLens textDocument/colorPresentation   
result     CodeLens[] | null            ColorPresentation[]   

type                                                                       \
method                   textDocument/completion textDocument/declaration   
result  CompletionItem[] | CompletionList | null         Location[] |null   

type    ...                   ➡️                        \
method  ... textDocument/didSave textDocument/willSave   
result  ...                 None                  None   

type                                                                     \
method workspace/didChangeConfiguration workspace/didChangeWatchedFiles   
result                             None                            None   

type                                                 ➡️ ⬅️              ⬅️  \
method workspace/didChangeWorkspaceFolders $/cancelRequest telemetry/event   
result                                None            None            None   

type                                                                         
method textDocument/publishDiagnostics window/logMessage window/showMessage  
result                            None              None               None  

[1 rows x 50 columns]

The remaining things look tedious to parse, but doable. Perhaps there is a better way...

## What's missing?

In [16]:
df[df["params_schema"].apply(lambda x: x is None)][["_raw_params", "params", "params_schema"]]

_raw_params  \
type  method                                                                                   
↩️    codeLens/resolve                                                            `CodeLens`   
      completionItem/resolve                                                `CompletionItem`   
      documentLink/resolve                                                    `DocumentLink`   
      textDocument/colorPresentation            `ColorPresentationParams` defined as follows   
      textDocument/documentColor                    `DocumentColorParams` defined as follows   
      textDocument/foldingRange                      `FoldingRangeParams` defined as follows   
↪️    workspace/configuration                       `ConfigurationParams` defined as follows   
➡️    workspace/didChangeWorkspaceFolders  `DidChangeWorkspaceFoldersParams` defined as f...   
➡️ ⬅️ $/cancelRequest                                     `CancelParams` defined as follows:   

                                                                    params  \
type  method                                                                 
↩️    codeLens/resolve                                            CodeLens   
      completionItem/resolve                                CompletionItem   
      documentLink/resolve                                    DocumentLink   
      textDocument/colorPresentation               ColorPresentationParams   
      textDocument/documentColor                       DocumentColorParams   
      textDocument/foldingRange                         FoldingRangeParams   
↪️    workspace/configuration                          ConfigurationParams   
➡️    workspace/didChangeWorkspaceFolders  DidChangeWorkspaceFoldersParams   
➡️ ⬅️ $/cancelRequest                                         CancelParams   

                                          params_schema  
type  method                                             
↩️    codeLens/resolve                             None  
      completionItem/resolve                       None  
      documentLink/resolve                         None  
      textDocument/colorPresentation               None  
      textDocument/documentColor                   None  
      textDocument/foldingRange                    None  
↪️    workspace/configuration                      None  
➡️    workspace/didChangeWorkspaceFolders          None  
➡️ ⬅️ $/cancelRequest                              None

In [17]:
df[df["result"].apply(lambda x: x is None)][~df["_raw_result"].isnull()][["_raw_result", "result"]]

/home/weg/Documents/projects/lsp-json-schema/envs/lsp-json-schema/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,,_raw_result,result
type,method,,


# `protocol-schema.ts`

A straight parsing of `protocol.ts` doesn't yield enough information for the generator to create all the types needed. Before we look any further at brute-forcing a solution, let's create a TypeScript file that includes everything we might need. First, as the feature titles are incosistently narrative, let's make some titles derived from the method names.

In [18]:
def method_title(m):
    name = "".join([
        b[0].upper() + b[1:]
        for b in m.split('/')
        if b != '$'
    ])
    return name
        
df["ns_title"] = list(df.reset_index()['method'].apply(method_title))
df[["ns_title"]].T

type                   ↩️                                              \
method   codeLens/resolve completionItem/resolve documentLink/resolve   
ns_title  CodeLensResolve  CompletionItemResolve  DocumentLinkResolve   

type                                                                          \
method    initialize  shutdown textDocument/codeAction textDocument/codeLens   
ns_title  Initialize  Shutdown  TextDocumentCodeAction  TextDocumentCodeLens   

type                                                             \
method   textDocument/colorPresentation textDocument/completion   
ns_title  TextDocumentColorPresentation  TextDocumentCompletion   

type                               ...                   ➡️  \
method   textDocument/declaration  ... textDocument/didSave   
ns_title  TextDocumentDeclaration  ...  TextDocumentDidSave   

type                                                             \
method   textDocument/willSave workspace/didChangeConfiguration   
ns_title  TextDocumentWillSave  WorkspaceDidChangeConfiguration   

type                                                                          \
method   workspace/didChangeWatchedFiles workspace/didChangeWorkspaceFolders   
ns_title  WorkspaceDidChangeWatchedFiles  WorkspaceDidChangeWorkspaceFolders   

type               ➡️ ⬅️              ⬅️                                  \
method   $/cancelRequest telemetry/event textDocument/publishDiagnostics   
ns_title   CancelRequest  TelemetryEvent  TextDocumentPublishDiagnostics   

type                                           
method   window/logMessage window/showMessage  
ns_title  WindowLogMessage  WindowShowMessage  

[1 rows x 50 columns]

As many of the results are unions, let's apply some string manipulation to put everything from `main.ts` in a namespace.

In [19]:
def ns_result(r):
    if r is None:
        return None
    try:
        r = re.sub(r'\b([A-Z])', 'proto.\\1', r)
    except Exception as err:
        print(err, r)
    return r
df["ns_result"] = df["result"].apply(ns_result)

This is not pretty, but gets the job done.

In [20]:
protocol_schema_ts = jinja2.Template("""
{% macro feature(row) -%}_{{ row.ns_title }}Feature{%- endmacro %}
{% macro request(row) -%}_{{ row.ns_title }}Request{%- endmacro %}
{% macro result(row) -%}_{{ row.ns_title }}Response{%- endmacro %}

import {RequestMessage} from 'vscode-jsonrpc';
import {ResponseMessage, ResponseErrorLiteral} from 'vscode-jsonrpc/lib/messages';
import * as proto from './main';

// this didn't get exported anywhere

export interface CancelParams {
    /**
     * The request id to cancel.
     */
    id: number | string;
}

// couldn't find this either

export interface _ErrorResponse extends Omit<ResponseMessage, 'result'>{
    error: ResponseErrorLiteral<any>;
}

export type _AnyFeature = {% for row in rows %}
    | {{ feature(row) }}{% endfor %};

export type _AnyRequest = {% for row in rows %}
    | {{ request(row) }}{% endfor %};

export type _AnyResponse = {% for row in rows %}
    {% if row.result %}| {{ result(row) }}{% endif %}{% endfor %};

{% for row in rows %}
export interface {{ request(row) }} extends RequestMessage {
    method: '{{ row.method }}';
    {%- if row.params %}
    params: {% if row.params == 'CancelParams' %}CancelParams{% else %}proto.{{ row.params }}{% endif %};
    {%- endif %}
}

{% if row.result %}
export interface {{ result(row) }} extends Omit<ResponseMessage, 'error'> {
    result: {{ row.ns_result }};
}
{% endif %}

export interface {{ feature(row) }} {
    request: {{ request(row) }};
    {% if row.result %}response: {{ result(row) }} | _ErrorResponse;
    {% endif %}
}
{% endfor %}

""").render(rows=df.reset_index().to_dict(orient="records"), ns_result=ns_result)

pathlib.Path("vscode-languageserver-node/protocol/src/protocol-schema.ts").write_text(protocol_schema_ts);

Our generated code probably isn't very pretty. I don't event want to look, when we can make it... `prettier`.

In [21]:
!wc vscode-languageserver-node/protocol/src/protocol-schema.ts
!cd vscode-languageserver-node && ./node_modules/.bin/prettier --write protocol/src/protocol-schema.ts
!wc vscode-languageserver-node/protocol/src/protocol-schema.ts

  947  1759 25075 vscode-languageserver-node/protocol/src/protocol-schema.ts
protocol/src/protocol-schema.tsprotocol/src/protocol-schema.ts 344ms
  811  1764 24007 vscode-languageserver-node/protocol/src/protocol-schema.ts


Now, let's create a new schema, derived from out schema-targeted source.

In [22]:
!rm generated.protocol.schema.json || echo "nothing to remove yet!"
!cd vscode-languageserver-node/protocol && ../node_modules/.bin/ts-json-schema-generator -c --path './src/protocol-schema.ts' --type '_AnyFeature' > ../../generated.protocol.schema.json
!wc generated.protocol.schema.json

  7958  20005 236588 generated.protocol.schema.json


Provided that worked, we should now be able to parse it.

In [23]:
generated_schema = json.loads(pathlib.Path("generated.protocol.schema.json").read_text())
# synthetic_schema["definitions"].update(naive_schema["definitions"])
generated_schema["definitions"] = dict(sorted(generated_schema["definitions"].items()))
len(generated_schema["definitions"]), "definitions"

(303, 'definitions')

And validate it

In [24]:
validator = jsonschema.validators.Draft7Validator(generated_schema)

Seems okay!

Let's re-apply what we've learned. We can now expect to find all of the request params under our made-up types.

In [25]:
df["params_schema"] = df["ns_title"].apply(
    lambda x: generated_schema["definitions"][f"_{x}Request"]["properties"]["params"] if f"_{x}Request" in generated_schema["definitions"] else None
)
df[["params", "params_schema"]].T

type                                                          ↩️  \
method                                          codeLens/resolve   
params                                                  CodeLens   
params_schema  {'$ref': '#/definitions/CodeLens', 'descriptio...   

type                                                              \
method                                    completionItem/resolve   
params                                            CompletionItem   
params_schema  {'$ref': '#/definitions/CompletionItem', 'desc...   

type                                                              \
method                                      documentLink/resolve   
params                                              DocumentLink   
params_schema  {'$ref': '#/definitions/DocumentLink', 'descri...   

type                                                              \
method                                                initialize   
params                                          InitializeParams   
params_schema  {'$ref': '#/definitions/InitializeParams', 'de...   

type                                                    \
method                                        shutdown   
params                                            None   
params_schema  {'description': 'The method's params.'}   

type                                                              \
method                                   textDocument/codeAction   
params                                          CodeActionParams   
params_schema  {'$ref': '#/definitions/CodeActionParams', 'de...   

type                                                              \
method                                     textDocument/codeLens   
params                                            CodeLensParams   
params_schema  {'$ref': '#/definitions/CodeLensParams', 'desc...   

type                                                              \
method                            textDocument/colorPresentation   
params                                   ColorPresentationParams   
params_schema  {'$ref': '#/definitions/ColorPresentationParam...   

type                                                              \
method                                   textDocument/completion   
params                                          CompletionParams   
params_schema  {'$ref': '#/definitions/CompletionParams', 'de...   

type                                                              ...  \
method                                  textDocument/declaration  ...   
params                                TextDocumentPositionParams  ...   
params_schema  {'$ref': '#/definitions/TextDocumentPositionPa...  ...   

type                                                          ➡️  \
method                                      textDocument/didSave   
params                                 DidSaveTextDocumentParams   
params_schema  {'$ref': '#/definitions/DidSaveTextDocumentPar...   

type                                                              \
method                                     textDocument/willSave   
params                                WillSaveTextDocumentParams   
params_schema  {'$ref': '#/definitions/WillSaveTextDocumentPa...   

type                                                              \
method                          workspace/didChangeConfiguration   
params                              DidChangeConfigurationParams   
params_schema  {'$ref': '#/definitions/DidChangeConfiguration...   

type                                                              \
method                           workspace/didChangeWatchedFiles   
params                               DidChangeWatchedFilesParams   
params_schema  {'$ref': '#/definitions/DidChangeWatchedFilesP...   

type                                                              \
method                       workspace/didChangeWorkspaceFolders   
params                           DidChangeWorkspaceFoldersPara

Similarly, all of the results.

In [26]:
df["result_schema"] = df["ns_title"].apply(
    lambda x: generated_schema["definitions"][f"_{x}Response"]["properties"]["result"] if f"_{x}Response" in generated_schema["definitions"] else None
)
df[["result", "result_schema"]].T

type                                                          ↩️  \
method                                          codeLens/resolve   
result                                                  CodeLens   
result_schema  {'anyOf': [{'type': 'string'}, {'type': 'numbe...   

type                                                              \
method                                    completionItem/resolve   
result                                            CompletionItem   
result_schema  {'anyOf': [{'type': 'string'}, {'type': 'numbe...   

type                                                              \
method                                      documentLink/resolve   
result                                              DocumentLink   
result_schema  {'anyOf': [{'type': 'string'}, {'type': 'numbe...   

type                                                              \
method                                                initialize   
result                                          InitializeResult   
result_schema  {'anyOf': [{'type': 'string'}, {'type': 'numbe...   

type                                                              \
method                                                  shutdown   
result                                                      null   
result_schema  {'anyOf': [{'type': 'string'}, {'type': 'numbe...   

type                                                              \
method                                   textDocument/codeAction   
result                           (Command | CodeAction)[] | null   
result_schema  {'anyOf': [{'items': {'anyOf': [{'$ref': '#/de...   

type                                                              \
method                                     textDocument/codeLens   
result                                         CodeLens[] | null   
result_schema  {'anyOf': [{'items': {'$ref': '#/definitions/C...   

type                                                              \
method                            textDocument/colorPresentation   
result                                       ColorPresentation[]   
result_schema  {'anyOf': [{'type': 'string'}, {'type': 'numbe...   

type                                                              \
method                                   textDocument/completion   
result                  CompletionItem[] | CompletionList | null   
result_schema  {'anyOf': [{'items': {'$ref': '#/definitions/C...   

type                                                              ...  \
method                                  textDocument/declaration  ...   
result                                          Location[] |null  ...   
result_schema  {'anyOf': [{'items': {'$ref': '#/definitions/L...  ...   

type                            ➡️                        \
method        textDocument/didSave textDocument/willSave   
result                        None                  None   
result_schema                 None                  None   

type                                            \
method        workspace/didChangeConfiguration   
result                                    None   
result_schema                             None   

type                                           \
method        workspace/didChangeWatchedFiles   
result                                   None   
result_schema                            None   

type                                                        ➡️ ⬅️  \
method        workspace/didChangeWorkspaceFolders $/cancelRequest   
result                                       None            None   
result_schema                                None            None   

type                       ⬅️                                  \
method        telemetry/event textDocument/publishDiagnostics   
result                   None                            None   
result_schema            None                            None   

type                                                
method        window

## What's missing now?

In [27]:
missing_params_from_generated_schema = df[df["params_schema"].isnull()][["_raw_params", "params", "params_schema"]]
missing_params_from_generated_schema

,,_raw_params,params,params_schema
type,method,,,


In [28]:
missing_results_from_generated_schema = df[df["result"].isnull()][~df["_raw_result"].isnull()][["_raw_result", "result", "result_schema"]]
missing_results_from_generated_schema

/home/weg/Documents/projects/lsp-json-schema/envs/lsp-json-schema/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,,_raw_result,result,result_schema
type,method,,,


In [29]:
assert not len(missing_params_from_generated_schema) and not len(missing_results_from_generated_schema)

Looks like we got it all.

# Validating things
First, let's validate something small: just a request to `initialize`.

In [30]:
def is_it_valid(name, instance):
    subschema = copy.deepcopy(generated_schema)
    subschema["$ref"] = f"#/definitions/{name}"
    jsonschema.validators.Draft7Validator(subschema).validate(instance)
    print(f"yep, it's a valid {name}.")

In [31]:
def is_it_invalid(name, instance, cause):
    with pytest.raises(jsonschema.ValidationError,  match=cause):
        try:
            is_it_valid(name, instance)
        except jsonschema.ValidationError as err:
            print(err.message)
            raise err
    print(f"oh no, it's not a valid {name}!")

In [32]:
init_params_good = {
    "capabilities": {},
    "processId": 1,
    "workspaceFolders": [],
    "rootUri": "file:///"
}

In [33]:
is_it_valid("InitializeParams", init_params_good)

yep, it's a valid InitializeParams.


It should catch something that's bad, like providing `capabilities`.

In [34]:
init_params_bad = copy.deepcopy(init_params_good)
init_params_bad.pop("capabilities")
is_it_invalid("InitializeParams", init_params_bad, "'capabilities' is a required property")

'capabilities' is a required property
oh no, it's not a valid InitializeParams!


The same approach could be used for any of the "real" types. However, the default schema points at `_AnyFeature`, which allows us to validate a request/response pair. While this is a side-effect of trying to generate a minimal schema (ha), it is nonetheless useful. Pretty much the simplest `initialize` exchange would be:

In [35]:
init_exchange_good = {
    "request": {
        "jsonrpc": "2.0",
        "id": 0,
        "method": "initialize",
        "params": init_params_good
    },
    "response": {
        "jsonrpc": "2.0",
        "id": 0,
        "result": {
            "capabilities": {},
            "serverInfo": {"name": "test"}
        }
    }
}

In [36]:
is_it_valid("_AnyFeature", init_exchange_good)

yep, it's a valid _AnyFeature.


But we might explicitly care if it's a good `_InitializeFeature`

In [37]:
is_it_valid("_InitializeFeature", init_exchange_good)

yep, it's a valid _InitializeFeature.


Imagine we had forgotten the `workspaceFolders`...

In [38]:
init_exchange_bad = copy.deepcopy(init_exchange_good)
init_exchange_bad["request"]["params"].pop("workspaceFolders")
is_it_invalid("_AnyFeature", init_exchange_bad, "is not valid")

{'request': {'jsonrpc': '2.0', 'id': 0, 'method': 'initialize', 'params': {'capabilities': {}, 'processId': 1, 'rootUri': 'file:///'}}, 'response': {'jsonrpc': '2.0', 'id': 0, 'result': {'capabilities': {}, 'serverInfo': {'name': 'test'}}}} is not valid under any of the given schemas
oh no, it's not a valid _AnyFeature!


Unfortunately, that error isn't _particularly_ helpful in telling us exactly _what_ went wrong, but it's probable that the schema could be restructured in such as way as to give better feedback.

# Remaining Challenges and Opportunities
- spec
    - if the specification format stays in Markdown
        - a formatter, e.g. [prettier](https://github.com/prettier/prettier) 
            - would aid in keeping the formatting consistent
        - a link checker
            - would aid in keeping the document discoverable
    - if an alternative format would be considered
        - an RFC-style spec
            - would aid in submission to an appropriate standards body (e.g. W3C, IETF)
        - a modular spec definition (e.g. XMPP's [XEP](https://xmpp.org/extensions/))
            - would allow for a more fine-grained discussion
- generator and/or reference implementation
    - the use of the `namespace x {const y0 = z0;const y1 = z1;}` pattern is relatively opaque to the generator
        - logically, this makes sense, as what's to really differentiate `1` from `1`, just in a different scheme?
        - but it would be really nice to talk about the names
- do more with the schema
    - it would be interesting to start wiring this up to [quickcheck](http://hackage.haskell.org/package/QuickCheck)-derived testing suite
    - ...or an interactive protocol documentation mode, perhaps like [rjsf](https://rjsf-team.github.io/react-jsonschema-form/)

# The Schema
For posterity, here's the whole thing. It should also be available as [JSON](./generated.protocol.schema.json) or [YAML](./generated.protocol.schema.yml).

In [39]:
yaml_schema = pathlib.Path("generated.protocol.schema.yml")
yaml_schema.write_text(yaml.safe_dump(generated_schema))
!./vscode-languageserver-node/node_modules/.bin/prettier --write {yaml_schema}
IPython.display.Markdown(f"""```yaml\n{yaml_schema.read_text()}\n```""")

generated.protocol.schema.ymlgenerated.protocol.schema.yml 1359ms


```yaml
$ref: "#/definitions/_AnyFeature"
$schema: http://json-schema.org/draft-07/schema#
definitions:
  ApplyWorkspaceEditParams:
    additionalProperties: false
    description: The parameters passed via a apply workspace edit request.
    properties:
      edit:
        $ref: "#/definitions/WorkspaceEdit"
        description: The edits to apply.
      label:
        description: "An optional label of the workspace edit. This label is

          presented in the user interface for example on an undo

          stack to undo the workspace edit."
        type: string
    required:
      - edit
    type: object
  ApplyWorkspaceEditResponse:
    additionalProperties: false
    description: A response returned from the apply workspace edit request.
    properties:
      applied:
        description: Indicates whether the edit was applied or not.
        type: boolean
      failedChange:
        description:
          "Depending on the client's failure handling strategy `failedChange`
          might

          contain the index of the change that failed. This property is only available

          if the client signals a `failureHandlingStrategy` in its client capabilities."
        type: number
      failureReason:
        description:
          "An optional textual description for why the edit was not applied.

          This may be used by the server for diagnostic logging or to provide

          a suitable error for a request that triggered the edit."
        type: string
    required:
      - applied
    type: object
  CancelParams:
    additionalProperties: false
    properties:
      id:
        description: The request id to cancel.
        type:
          - number
          - string
    required:
      - id
    type: object
  ClientCapabilities:
    additionalProperties: false
    properties:
      dynamicRegistration:
        description:
          "Whether implementation supports dynamic registration for selection
          range providers. If this is set to `true`

          the client supports the new `SelectionRangeRegistrationOptions` return value
          for the corresponding server

          capability as well."
        type: boolean
      experimental:
        description: Experimental client capabilities.
      textDocument:
        $ref: "#/definitions/TextDocumentClientCapabilities"
        description: Text document specific client capabilities.
      window:
        description: Window specific client capabilities.
        type: object
      workspace:
        $ref: "#/definitions/WorkspaceClientCapabilities"
        additionalProperties: false
        description: Workspace specific client capabilities.
        properties:
          configuration:
            description: The client supports `workspace/configuration` requests.
            type: boolean
          workspaceFolders:
            description: The client has support for workspace folders
            type: boolean
        type: object
    type: object
  CodeAction:
    additionalProperties: false
    description:
      "A code action represents a change that can be performed in code,\
      \ e.g. to fix a problem or\r\nto refactor code.\r\n\r\nA CodeAction must set\
      \ either `edit` and/or a `command`. If both are supplied, the `edit` is applied\
      \ first, then the `command` is executed."
    properties:
      command:
        $ref: "#/definitions/Command"
        description:
          "A command this code action executes. If a code action\r\nprovides\
          \ a edit and a command, first the edit is\r\nexecuted and then the command."
      diagnostics:
        description: The diagnostics that this code action resolves.
        items:
          $ref: "#/definitions/Diagnostic"
        type: array
      edit:
        $ref: "#/definitions/WorkspaceEdit"
        description: The workspace edit this code action performs.
      isPreferred:
        description:
          "Marks this as a preferred action. Preferred actions are used\
          \ by the `auto fix` command and can be targeted\r\nby keybindings.\r\n\r\
          \nA quick fix should be marked preferred if it properly addresses the underlying\
          \ error.\r\nA refactoring should be marked preferred if it is the most reasonable\
          \ choice of actions to take."
        type: boolean
      kind:
        $ref: "#/definitions/CodeActionKind"
        description: "The kind of the code action.\r\n\r\nUsed to filter code actions."
      title:
        description: A short, human-readable, title for this code action.
        type: string
    required:
      - title
    type: object
  CodeActionClientCapabilities:
    additionalProperties: false
    description: The Client Capabilities of a [CodeActionRequest](#CodeActionRequest).
    properties:
      codeActionLiteralSupport:
        additionalProperties: false
        description: "The client support code action literals as a valid

          response of the `textDocument/codeAction` request."
        properties:
          codeActionKind:
            additionalProperties: false
            description:
              "The code action kind is support with the following value

              set."
            properties:
              valueSet:
                description:
                  "The code action kind values the client supports. When
                  this

                  property exists the client also guarantees that it will

                  handle values outside its set gracefully and falls back

                  to a default value when unknown."
                items:
                  $ref: "#/definitions/CodeActionKind"
                type: array
            required:
              - valueSet
            type: object
        required:
          - codeActionKind
        type: object
      dynamicRegistration:
        description: Whether code action supports dynamic registration.
        type: boolean
      isPreferredSupport:
        description: Whether code action supports the `isPreferred` property.
        type: boolean
    type: object
  CodeActionContext:
    additionalProperties: false
    description:
      "Contains additional diagnostic information about the context in\
      \ which\r\na [code action](#CodeActionProvider.provideCodeActions) is run. \n\
      \ The CodeActionContext namespace provides helper functions to work with\r\n\
      [CodeActionContext](#CodeActionContext) literals."
    properties:
      diagnostics:
        description:
          "An array of diagnostics known on the client side overlapping\
          \ the range provided to the\r\n`textDocument/codeAction` request. They are\
          \ provied so that the server knows which\r\nerrors are currently presented\
          \ to the user for the given range. There is no guarantee\r\nthat these accurately\
          \ reflect the error state of the resource. The primary parameter\r\nto compute\
          \ code actions is the provided range."
        items:
          $ref: "#/definitions/Diagnostic"
        type: array
      only:
        description:
          "Requested kind of actions to return.\r\n\r\nActions not of this\
          \ kind are filtered out by the client before being shown. So servers\r\n\
          can omit computing them."
        items:
          $ref: "#/definitions/CodeActionKind"
        type: array
    required:
      - diagnostics
    type: object
  CodeActionKind:
    description:
      "The kind of a code action.\r\n\r\nKinds are a hierarchical list\
      \ of identifiers separated by `.`, e.g. `\"refactor.extract.function\"`.\r\n\
      \r\nThe set of kinds is open and client needs to announce the kinds it supports\
      \ to the server during\r\ninitialization. \n A set of predefined code action\
      \ kinds"
    type: string
  CodeActionOptions:
    additionalProperties: false
    description: Provider options for a [CodeActionRequest](#CodeActionRequest).
    properties:
      codeActionKinds:
        description: "CodeActionKinds that this server may return.


          The list of kinds may be generic, such as `CodeActionKind.Refactor`, or
          the server

          may list out every specific kind they provide."
        items:
          $ref: "#/definitions/CodeActionKind"
        type: array
      workDoneProgress:
        type: boolean
    type: object
  CodeActionParams:
    additionalProperties: false
    description: The parameters of a [CodeActionRequest](#CodeActionRequest).
    properties:
      context:
        $ref: "#/definitions/CodeActionContext"
        description: Context carrying additional information.
      partialResultToken:
        $ref: "#/definitions/ProgressToken"
        description:
          "An optional token that a server can use to report partial results
          (e.g. streaming) to

          the client."
      range:
        $ref: "#/definitions/Range"
        description: The range for which the command was invoked.
      textDocument:
        $ref: "#/definitions/TextDocumentIdentifier"
        description: The document in which the command was invoked.
      workDoneToken:
        $ref: "#/definitions/ProgressToken"
        description: An optional token that a server can use to report work done progress.
    required:
      - textDocument
      - range
      - context
    type: object
  CodeLens:
    additionalProperties: false
    description:
      "A code lens represents a [command](#Command) that should be shown\
      \ along with\r\nsource text, like the number of references, a way to run tests,\
      \ etc.\r\n\r\nA code lens is _unresolved_ when no command is associated to it.\
      \ For performance\r\nreasons the creation of a code lens and resolving should\
      \ be done to two stages. \n The CodeLens namespace provides helper functions\
      \ to work with\r\n[CodeLens](#CodeLens) literals."
    properties:
      command:
        $ref: "#/definitions/Command"
        description: The command this code lens represents.
      data:
        description:
          "An data entry field that is preserved on a code lens item between\r\
          \na [CodeLensRequest](#CodeLensRequest) and a [CodeLensResolveRequest]\r\
          \n(#CodeLensResolveRequest)"
      range:
        $ref: "#/definitions/Range"
        description:
          The range in which this code lens is valid. Should only span
          a single line.
    required:
      - range
    type: object
  CodeLensClientCapabilities:
    additionalProperties: false
    description: The client capabilities  of a [CodeLensRequest](#CodeLensRequest).
    properties:
      dynamicRegistration:
        description: Whether code lens supports dynamic registration.
        type: boolean
    type: object
  CodeLensOptions:
    additionalProperties: false
    description: Code Lens provider options of a [CodeLensRequest](#CodeLensRequest).
    properties:
      resolveProvider:
        description: Code lens has a resolve provider as well.
        type: boolean
      workDoneProgress:
        type: boolean
    type: object
  CodeLensParams:
    additionalProperties: false
    description: The parameters of a [CodeLensRequest](#CodeLensRequest).
    properties:
      partialResultToken:
        $ref: "#/definitions/ProgressToken"
        description:
          "An optional token that a server can use to report partial results
          (e.g. streaming) to

          the client."
      textDocument:
        $ref: "#/definitions/TextDocumentIdentifier"
        description: The document to request code lens for.
      workDoneToken:
        $ref: "#/definitions/ProgressToken"
        description: An optional token that a server can use to report work done progress.
    required:
      - textDocument
    type: object
  Color:
    additionalProperties: false
    description:
      "Represents a color in RGBA space. \n The Color namespace provides\
      \ helper functions to work with\r\n[Color](#Color) literals."
    properties:
      alpha:
        description: The alpha component of this color in the range [0-1].
        type: number
      blue:
        description: The blue component of this color in the range [0-1].
        type: number
      green:
        description: The green component of this color in the range [0-1].
        type: number
      red:
        description: The red component of this color in the range [0-1].
        type: number
    required:
      - red
      - green
      - blue
      - alpha
    type: object
  ColorInformation:
    additionalProperties: false
    description:
      "Represents a color range from a document. \n The ColorInformation\
      \ namespace provides helper functions to work with\r\n[ColorInformation](#ColorInformation)\
      \ literals."
    properties:
      color:
        $ref: "#/definitions/Color"
        description: The actual color value for this color range.
      range:
        $ref: "#/definitions/Range"
        description: The range in the document where this color appers.
    required:
      - range
      - color
    type: object
  ColorPresentation:
    additionalProperties: false
    description:
      "The Color namespace provides helper functions to work with\r\n[ColorPresentation](#ColorPresentation)\
      \ literals."
    properties:
      additionalTextEdits:
        description:
          "An optional array of additional [text edits](#TextEdit) that\
          \ are applied when\r\nselecting this color presentation. Edits must not\
          \ overlap with the main [edit](#ColorPresentation.textEdit) nor with themselves."
        items:
          $ref: "#/definitions/TextEdit"
        type: array
      label:
        description:
          "The label of this color presentation. It will be shown on the\
          \ color\r\npicker header. By default this is also the text that is inserted\
          \ when selecting\r\nthis color presentation."
        type: string
      textEdit:
        $ref: "#/definitions/TextEdit"
        description:
          "An [edit](#TextEdit) which is applied to a document when selecting\r\
          \nthis presentation for the color.  When `falsy` the [label](#ColorPresentation.label)\r\
          \nis used."
    required:
      - label
    type: object
  ColorPresentationParams:
    additionalProperties: false
    description: Parameters for a [ColorPresentationRequest](#ColorPresentationRequest).
    properties:
      color:
        $ref: "#/definitions/Color"
        description: The color to request presentations for.
      partialResultToken:
        $ref: "#/definitions/ProgressToken"
        description:
          "An optional token that a server can use to report partial results
          (e.g. streaming) to

          the client."
      range:
        $ref: "#/definitions/Range"
        description: The range where the color would be inserted. Serves as a context.
      textDocument:
        $ref: "#/definitions/TextDocumentIdentifier"
        description: The text document.
      workDoneToken:
        $ref: "#/definitions/ProgressToken"
        description: An optional token that a server can use to report work done progress.
    required:
      - textDocument
      - color
      - range
    type: object
  Command:
    additionalProperties: false
    description:
      "Represents a reference to a command. Provides a title which\r\n\
      will be used to represent a command in the UI and, optionally,\r\nan array of\
      \ arguments which will be passed to the command handler\r\nfunction when invoked.\
      \ \n The Command namespace provides helper functions to work with\r\n[Command](#Command)\
      \ literals."
    properties:
      arguments:
        description: "Arguments that the command handler should be\r\ninvoked with."
        items: {}
        type: array
      command:
        description: The identifier of the actual command handler.
        type: string
      title:
        description: Title of the command, like `save`.
        type: string
    required:
      - title
      - command
    type: object
  CompletionClientCapabilities:
    additionalProperties: false
    description: Completion client capabilities
    properties:
      completionItem:
        additionalProperties: false
        description:
          "The client supports the following `CompletionItem` specific

          capabilities."
        properties:
          commitCharactersSupport:
            description: Client supports commit characters on a completion item.
            type: boolean
          deprecatedSupport:
            description: Client supports the deprecated property on a completion item.
            type: boolean
          documentationFormat:
            description:
              "Client supports the follow content formats for the documentation

              property. The order describes the preferred format of the client."
            items:
              $ref: "#/definitions/MarkupKind"
            type: array
          preselectSupport:
            description: Client supports the preselect property on a completion item.
            type: boolean
          snippetSupport:
            description: "Client supports snippets as insert text.


              A snippet can define tab stops and placeholders with `$1`, `$2`

              and `${3:foo}`. `$0` defines the final tab stop, it defaults to

              the end of the snippet. Placeholders with equal identifiers are linked,

              that is typing in one will update others too."
            type: boolean
          tagSupport:
            additionalProperties: false
            description:
              "Client supports the tag property on a completion item. Clients
              supporting

              tags have to handle unknown tags gracefully. Clients especially need
              to

              preserve unknown tags when sending a completion item back to the server
              in

              a resolve call."
            properties:
              valueSet:
                description: The tags supported by the client.
                items:
                  $ref: "#/definitions/CompletionItemTag"
                type: array
            required:
              - valueSet
            type: object
        type: object
      completionItemKind:
        additionalProperties: false
        properties:
          valueSet:
            description:
              "The completion item kind values the client supports. When
              this

              property exists the client also guarantees that it will

              handle values outside its set gracefully and falls back

              to a default value when unknown.


              If this property is not present the client only supports

              the completion items kinds from `Text` to `Reference` as defined in

              the initial version of the protocol."
            items:
              $ref: "#/definitions/CompletionItemKind"
            type: array
        type: object
      contextSupport:
        description:
          "The client supports to send additional context information for
          a

          `textDocument/completion` requestion."
        type: boolean
      dynamicRegistration:
        description: Whether completion supports dynamic registration.
        type: boolean
    type: object
  CompletionContext:
    additionalProperties: false
    description:
      Contains additional information about the context in which a completion
      request is triggered.
    properties:
      triggerCharacter:
        description:
          "The trigger character (a single character) that has trigger
          code complete.

          Is undefined if `triggerKind !== CompletionTriggerKind.TriggerCharacter`"
        type: string
      triggerKind:
        $ref: "#/definitions/CompletionTriggerKind"
        description: How the completion was triggered.
    required:
      - triggerKind
    type: object
  CompletionItem:
    additionalProperties: false
    description:
      "A completion item represents a text snippet that is\r\nproposed\
      \ to complete text that is being typed. \n The CompletionItem namespace provides\
      \ functions to deal with\r\ncompletion items."
    properties:
      additionalTextEdits:
        description:
          "An optional array of additional [text edits](#TextEdit) that\
          \ are applied when\r\nselecting this completion. Edits must not overlap\
          \ (including the same insert position)\r\nwith the main [edit](#CompletionItem.textEdit)\
          \ nor with themselves.\r\n\r\nAdditional text edits should be used to change\
          \ text unrelated to the current cursor position\r\n(for example adding an\
          \ import statement at the top of the file if the completion item will\r\n\
          insert an unqualified type)."
        items:
          $ref: "#/definitions/TextEdit"
        type: array
      command:
        $ref: "#/definitions/Command"
        description:
          "An optional [command](#Command) that is executed *after* inserting\
          \ this completion. *Note* that\r\nadditional modifications to the current\
          \ document should be described with the\r\n[additionalTextEdits](#CompletionItem.additionalTextEdits)-property."
      commitCharacters:
        description:
          "An optional set of characters that when pressed while this completion\
          \ is active will accept it first and\r\nthen type that character. *Note*\
          \ that all commit characters should have `length=1` and that superfluous\r\
          \ncharacters will be ignored."
        items:
          type: string
        type: array
      data:
        description:
          "An data entry field that is preserved on a completion item between\r\
          \na [CompletionRequest](#CompletionRequest) and a [CompletionResolveRequest]\r\
          \n(#CompletionResolveRequest)"
      deprecated:
        description: Indicates if this item is deprecated.
        type: boolean
      detail:
        description:
          "A human-readable string with additional information\r\nabout\
          \ this item, like type or symbol information."
        type: string
      documentation:
        anyOf:
          - type: string
          - $ref: "#/definitions/MarkupContent"
        description: A human-readable string that represents a doc-comment.
      filterText:
        description:
          "A string that should be used when filtering a set of\r\ncompletion\
          \ items. When `falsy` the [label](#CompletionItem.label)\r\nis used."
        type: string
      insertText:
        description:
          "A string that should be inserted into a document when selecting\r\
          \nthis completion. When `falsy` the [label](#CompletionItem.label)\r\nis\
          \ used.\r\n\r\nThe `insertText` is subject to interpretation by the client\
          \ side.\r\nSome tools might not take the string literally. For example\r\
          \nVS Code when code complete is requested in this example `con<cursor position>`\r\
          \nand a completion item with an `insertText` of `console` is provided it\r\
          \nwill only insert `sole`. Therefore it is recommended to use `textEdit`\
          \ instead\r\nsince it avoids additional client side interpretation."
        type: string
      insertTextFormat:
        $ref: "#/definitions/InsertTextFormat"
        description:
          "The format of the insert text. The format applies to both the\
          \ `insertText` property\r\nand the `newText` property of a provided `textEdit`."
      kind:
        $ref: "#/definitions/CompletionItemKind"
        description:
          "The kind of this completion item. Based of the kind\r\nan icon\
          \ is chosen by the editor."
      label:
        description:
          "The label of this completion item. By default\r\nalso the text\
          \ that is inserted when selecting\r\nthis completion."
        type: string
      preselect:
        description:
          "Select this item when showing.\r\n\r\n*Note* that only one completion\
          \ item can be selected and that the\r\ntool / client decides which item\
          \ that is. The rule is that the *first*\r\nitem of those that match best\
          \ is selected."
        type: boolean
      sortText:
        description:
          "A string that should be used when comparing this item\r\nwith\
          \ other items. When `falsy` the [label](#CompletionItem.label)\r\nis used."
        type: string
      tags:
        description: Tags for this completion item.
        items:
          $ref: "#/definitions/CompletionItemTag"
        type: array
      textEdit:
        $ref: "#/definitions/TextEdit"
        description:
          "An [edit](#TextEdit) which is applied to a document when selecting\r\
          \nthis completion. When an edit is provided the value of\r\n[insertText](#CompletionItem.insertText)\
          \ is ignored.\r\n\r\n*Note:* The text edit's range must be a [single line]\
          \ and it must contain the position\r\nat which completion has been requested."
    required:
      - label
    type: object
  CompletionItemKind:
    description: The kind of a completion entry.
    enum:
      - 1
      - 2
      - 3
      - 4
      - 5
      - 6
      - 7
      - 8
      - 9
      - 10
      - 11
      - 12
      - 13
      - 14
      - 15
      - 16
      - 17
      - 18
      - 19
      - 20
      - 21
      - 22
      - 23
      - 24
      - 25
    type: number
  CompletionItemTag:
    description:
      "Completion item tags are extra annotations that tweak the rendering\
      \ of a completion\r\nitem."
    enum:
      - 1
    type: number
  CompletionList:
    additionalProperties: false
    description:
      "Represents a collection of [completion items](#CompletionItem) to\
      \ be presented\r\nin the editor. \n The CompletionList namespace provides functions\
      \ to deal with\r\ncompletion lists."
    properties:
      isIncomplete:
        description:
          This list it not complete. Further typing results in recomputing
          this list.
        type: boolean
      items:
        description: The completion items.
        items:
          $ref: "#/definitions/CompletionItem"
        type: array
    required:
      - isIncomplete
      - items
    type: object
  CompletionOptions:
    additionalProperties: false
    description: Completion options.
    properties:
      allCommitCharacters:
        description:
          "The list of all possible characters that commit a completion.
          This field can be used

          if clients don't support individual commmit characters per completion item.
          See

          `ClientCapabilities.textDocument.completion.completionItem.commitCharactersSupport`"
        items:
          type: string
        type: array
      resolveProvider:
        description: "The server provides support to resolve additional

          information for a completion item."
        type: boolean
      triggerCharacters:
        description:
          "Most tools trigger completion request automatically without
          explicitly requesting

          it using a keyboard shortcut (e.g. Ctrl+Space). Typically they do so when
          the user

          starts to type an identifier. For example if the user types `c` in a JavaScript
          file

          code complete will automatically pop up present `console` besides others
          as a

          completion item. Characters that make up identifiers don't need to be listed
          here.


          If code complete should automatically be trigger on characters not being
          valid inside

          an identifier (for example `.` in JavaScript) list them in `triggerCharacters`."
        items:
          type: string
        type: array
      workDoneProgress:
        type: boolean
    type: object
  CompletionParams:
    additionalProperties: false
    description: Completion parameters
    properties:
      context:
        $ref: "#/definitions/CompletionContext"
        description:
          "The completion context. This is only available it the client
          specifies

          to send this using the client capability `textDocument.completion.contextSupport
          === true`"
      partialResultToken:
        $ref: "#/definitions/ProgressToken"
        description:
          "An optional token that a server can use to report partial results
          (e.g. streaming) to

          the client."
      position:
        $ref: "#/definitions/Position"
        description: The position inside the text document.
      textDocument:
        $ref: "#/definitions/TextDocumentIdentifier"
        description: The text document.
      workDoneToken:
        $ref: "#/definitions/ProgressToken"
        description: An optional token that a server can use to report work done progress.
    required:
      - position
      - textDocument
    type: object
  CompletionTriggerKind:
    description: How a completion was triggered
    enum:
      - 1
      - 2
      - 3
    type: number
  ConfigurationItem:
    additionalProperties: false
    properties:
      scopeUri:
        description: The scope to get the configuration section for.
        type: string
      section:
        description: The configuration section asked for.
        type: string
    type: object
  ConfigurationParams:
    additionalProperties: false
    description: The parameters of a configuration request.
    properties:
      items:
        items:
          $ref: "#/definitions/ConfigurationItem"
        type: array
    required:
      - items
    type: object
  CreateFile:
    additionalProperties: false
    description: Create file operation.
    properties:
      kind:
        description: A create
        enum:
          - create
        type: string
      options:
        $ref: "#/definitions/CreateFileOptions"
        description: Additional options
      uri:
        $ref: "#/definitions/DocumentUri"
        description: The resource to create.
    required:
      - kind
      - uri
    type: object
  CreateFileOptions:
    additionalProperties: false
    description: Options to create a file.
    properties:
      ignoreIfExists:
        description: Ignore if exists.
        type: boolean
      overwrite:
        description: Overwrite existing file. Overwrite wins over `ignoreIfExists`
        type: boolean
    type: object
  DeclarationClientCapabilities:
    additionalProperties: false
    description: Since 3.14.0
    properties:
      dynamicRegistration:
        description:
          "Whether declaration supports dynamic registration. If this is
          set to `true`

          the client supports the new `DeclarationRegistrationOptions` return value

          for the corresponding server capability as well."
        type: boolean
      linkSupport:
        description:
          The client supports additional metadata in the form of declaration
          links.
        type: boolean
    type: object
  DeclarationOptions:
    additionalProperties: false
    properties:
      workDoneProgress:
        type: boolean
    type: object
  DeclarationRegistrationOptions:
    additionalProperties: false
    properties:
      documentSelector:
        anyOf:
          - $ref: "#/definitions/DocumentSelector"
          - type: "null"
        description:
          "A document selector to identify the scope of the registration.
          If set to null

          the document selector provided on the client side will be used."
      id:
        description:
          "The id used to register the request. The id can be used to deregister

          the request again. See also Registration#id."
        type: string
      workDoneProgress:
        type: boolean
    required:
      - documentSelector
    type: object
  DefinitionClientCapabilities:
    additionalProperties: false
    description: Client Capabilities for a [DefinitionRequest](#DefinitionRequest).
    properties:
      dynamicRegistration:
        description: Whether definition supports dynamic registration.
        type: boolean
      linkSupport:
        description:
          The client supports additional metadata in the form of definition
          links.
        type: boolean
    type: object
  DefinitionOptions:
    additionalProperties: false
    description: Server Capabilities for a [DefinitionRequest](#DefinitionRequest).
    properties:
      workDoneProgress:
        type: boolean
    type: object
  DeleteFile:
    additionalProperties: false
    description: Delete file operation
    properties:
      kind:
        description: A delete
        enum:
          - delete
        type: string
      options:
        $ref: "#/definitions/DeleteFileOptions"
        description: Delete options.
      uri:
        $ref: "#/definitions/DocumentUri"
        description: The file to delete.
    required:
      - kind
      - uri
    type: object
  DeleteFileOptions:
    additionalProperties: false
    description: Delete file options
    properties:
      ignoreIfNotExists:
        description: Ignore the operation if the file doesn't exist.
        type: boolean
      recursive:
        description: Delete the content recursively if a folder is denoted.
        type: boolean
    type: object
  Diagnostic:
    additionalProperties: false
    description:
      "Represents a diagnostic, such as a compiler error or warning. Diagnostic\
      \ objects\r\nare only valid in the scope of a resource. \n The Diagnostic namespace\
      \ provides helper functions to work with\r\n[Diagnostic](#Diagnostic) literals."
    properties:
      code:
        description: The diagnostic's code, which usually appear in the user interface.
        type:
          - number
          - string
      message:
        description: The diagnostic's message. It usually appears in the user interface
        type: string
      range:
        $ref: "#/definitions/Range"
        description: The range at which the message applies
      relatedInformation:
        description:
          "An array of related diagnostic information, e.g. when symbol-names\
          \ within\r\na scope collide all definitions can be marked via this property."
        items:
          $ref: "#/definitions/DiagnosticRelatedInformation"
        type: array
      severity:
        $ref: "#/definitions/DiagnosticSeverity"
        description:
          "The diagnostic's severity. Can be omitted. If omitted it is\
          \ up to the\r\nclient to interpret diagnostics as error, warning, info or\
          \ hint."
      source:
        description:
          "A human-readable string describing the source of this\r\ndiagnostic,\
          \ e.g. 'typescript' or 'super lint'. It usually\r\nappears in the user interface."
        type: string
      tags:
        description: Additional metadata about the diagnostic.
        items:
          $ref: "#/definitions/DiagnosticTag"
        type: array
    required:
      - range
      - message
    type: object
  DiagnosticRelatedInformation:
    additionalProperties: false
    description:
      "Represents a related message and source code location for a diagnostic.\
      \ This should be\r\nused to point to code locations that cause or related to\
      \ a diagnostics, e.g when duplicating\r\na symbol in a scope. \n The DiagnosticRelatedInformation\
      \ namespace provides helper functions to work with\r\n[DiagnosticRelatedInformation](#DiagnosticRelatedInformation)\
      \ literals."
    properties:
      location:
        $ref: "#/definitions/Location"
        description: The location of this related diagnostic information.
      message:
        description: The message of this related diagnostic information.
        type: string
    required:
      - location
      - message
    type: object
  DiagnosticSeverity:
    description: The diagnostic's severity.
    enum:
      - 1
      - 2
      - 3
      - 4
    type: number
  DiagnosticTag:
    description: The diagnostic tags.
    enum:
      - 1
      - 2
    type: number
  DidChangeConfigurationClientCapabilities:
    additionalProperties: false
    properties:
      dynamicRegistration:
        description: Did change configuration notification supports dynamic registration.
        type: boolean
    type: object
  DidChangeConfigurationParams:
    additionalProperties: false
    description: The parameters of a change configuration notification.
    properties:
      settings:
        description: The actual changed settings
    required:
      - settings
    type: object
  DidChangeTextDocumentParams:
    additionalProperties: false
    description: The change text document notification's parameters.
    properties:
      contentChanges:
        description:
          "The actual content changes. The content changes describe single
          state changes

          to the document. So if there are two content changes c1 and c2 for a document

          in state S then c1 move the document to S' and c2 to S''."
        items:
          $ref: "#/definitions/TextDocumentContentChangeEvent"
        type: array
      textDocument:
        $ref: "#/definitions/VersionedTextDocumentIdentifier"
        description: "The document that did change. The version number points

          to the version after all provided content changes have

          been applied."
    required:
      - textDocument
      - contentChanges
    type: object
  DidChangeWatchedFilesClientCapabilities:
    additionalProperties: false
    properties:
      dynamicRegistration:
        description:
          "Did change watched files notification supports dynamic registration.
          Please note

          that the current protocol doesn't support static configuration for file
          changes

          from the server side."
        type: boolean
    type: object
  DidChangeWatchedFilesParams:
    additionalProperties: false
    description: The watched files change notification's parameters.
    properties:
      changes:
        description: The actual file events.
        items:
          $ref: "#/definitions/FileEvent"
        type: array
    required:
      - changes
    type: object
  DidChangeWorkspaceFoldersParams:
    additionalProperties: false
    description: The parameters of a `workspace/didChangeWorkspaceFolders` notification.
    properties:
      event:
        $ref: "#/definitions/WorkspaceFoldersChangeEvent"
        description: The actual workspace folder change event.
    required:
      - event
    type: object
  DidCloseTextDocumentParams:
    additionalProperties: false
    description: The parameters send in a close text document notification
    properties:
      textDocument:
        $ref: "#/definitions/TextDocumentIdentifier"
        description: The document that was closed.
    required:
      - textDocument
    type: object
  DidOpenTextDocumentParams:
    additionalProperties: false
    description: The parameters send in a open text document notification
    properties:
      textDocument:
        $ref: "#/definitions/TextDocumentItem"
        description: The document that was opened.
    required:
      - textDocument
    type: object
  DidSaveTextDocumentParams:
    additionalProperties: false
    description: The parameters send in a save text document notification
    properties:
      text:
        description:
          "Optional the content when saved. Depends on the includeText
          value

          when the save notification was requested."
        type: string
      textDocument:
        $ref: "#/definitions/VersionedTextDocumentIdentifier"
        description: The document that was closed.
    required:
      - textDocument
    type: object
  DocumentColorClientCapabilities:
    additionalProperties: false
    properties:
      dynamicRegistration:
        description:
          "Whether implementation supports dynamic registration. If this
          is set to `true`

          the client supports the new `DocumentColorRegistrationOptions` return value

          for the corresponding server capability as well."
        type: boolean
    type: object
  DocumentColorOptions:
    additionalProperties: false
    properties:
      resolveProvider:
        description: Code lens has a resolve provider as well.
        type: boolean
      workDoneProgress:
        type: boolean
    type: object
  DocumentColorParams:
    additionalProperties: false
    description: Parameters for a [DocumentColorRequest](#DocumentColorRequest).
    properties:
      partialResultToken:
        $ref: "#/definitions/ProgressToken"
        description:
          "An optional token that a server can use to report partial results
          (e.g. streaming) to

          the client."
      textDocument:
        $ref: "#/definitions/TextDocumentIdentifier"
        description: The text document.
      workDoneToken:
        $ref: "#/definitions/ProgressToken"
        description: An optional token that a server can use to report work done progress.
    required:
      - textDocument
    type: object
  DocumentColorRegistrationOptions:
    additionalProperties: false
    properties:
      documentSelector:
        anyOf:
          - $ref: "#/definitions/DocumentSelector"
          - type: "null"
        description:
          "A document selector to identify the scope of the registration.
          If set to null

          the document selector provided on the client side will be used."
      id:
        description:
          "The id used to register the request. The id can be used to deregister

          the request again. See also Registration#id."
        type: string
      resolveProvider:
        description: Code lens has a resolve provider as well.
        type: boolean
      workDoneProgress:
        type: boolean
    required:
      - documentSelector
    type: object
  DocumentFilter:
    anyOf:
      - additionalProperties: false
        properties:
          language:
            description: A language id, like `typescript`.
            type: string
          pattern:
            description: A glob pattern, like `*.{ts,js}`.
            type: string
          scheme:
            description: A Uri [scheme](#Uri.scheme), like `file` or `untitled`.
            type: string
        required:
          - language
        type: object
      - additionalProperties: false
        properties:
          language:
            description: A language id, like `typescript`.
            type: string
          pattern:
            description: A glob pattern, like `*.{ts,js}`.
            type: string
          scheme:
            description: A Uri [scheme](#Uri.scheme), like `file` or `untitled`.
            type: string
        required:
          - scheme
        type: object
      - additionalProperties: false
        properties:
          language:
            description: A language id, like `typescript`.
            type: string
          pattern:
            description: A glob pattern, like `*.{ts,js}`.
            type: string
          scheme:
            description: A Uri [scheme](#Uri.scheme), like `file` or `untitled`.
            type: string
        required:
          - pattern
        type: object
    description:
      "A document filter denotes a document by different properties like\n\
      the [language](#TextDocument.languageId), the [scheme](#Uri.scheme) of\nits\
      \ resource, or a glob-pattern that is applied to the [path](#TextDocument.fileName).\n\
      \nGlob patterns can have the following syntax:\n- `*` to match one or more characters\
      \ in a path segment\n- `?` to match on one character in a path segment\n- `**`\
      \ to match any number of path segments, including none\n- `{}` to group conditions\
      \ (e.g. `**\u200B/*.{ts,js}` matches all TypeScript and JavaScript files)\n\
      - `[]` to declare a range of characters to match in a path segment (e.g., `example.[0-9]`\
      \ to match on `example.0`, `example.1`, \u2026)\n- `[!...]` to negate a range\
      \ of characters to match in a path segment (e.g., `example.[!0-9]` to match\
      \ on `example.a`, `example.b`, but not `example.0`) \n The DocumentFilter namespace\
      \ provides helper functions to work with\n[DocumentFilter](#DocumentFilter)\
      \ literals."
  DocumentFormattingClientCapabilities:
    additionalProperties: false
    description: Client capabilities of a [DocumentFormattingRequest](#DocumentFormattingRequest).
    properties:
      dynamicRegistration:
        description: Whether formatting supports dynamic registration.
        type: boolean
    type: object
  DocumentFormattingOptions:
    additionalProperties: false
    description: Provider options for a [DocumentFormattingRequest](#DocumentFormattingRequest).
    properties:
      workDoneProgress:
        type: boolean
    type: object
  DocumentFormattingParams:
    additionalProperties: false
    description: The parameters of a [DocumentFormattingRequest](#DocumentFormattingRequest).
    properties:
      options:
        $ref: "#/definitions/FormattingOptions"
        description: The format options
      textDocument:
        $ref: "#/definitions/TextDocumentIdentifier"
        description: The document to format.
      workDoneToken:
        $ref: "#/definitions/ProgressToken"
        description: An optional token that a server can use to report work done progress.
    required:
      - textDocument
      - options
    type: object
  DocumentHighlight:
    additionalProperties: false
    description:
      "A document highlight is a range inside a text document which deserves\r\
      \nspecial attention. Usually a document highlight is visualized by changing\r\
      \nthe background color of its range. \n DocumentHighlight namespace to provide\
      \ helper functions to work with\r\n[DocumentHighlight](#DocumentHighlight) literals."
    properties:
      kind:
        $ref: "#/definitions/DocumentHighlightKind"
        description: The highlight kind, default is [text](#DocumentHighlightKind.Text).
      range:
        $ref: "#/definitions/Range"
        description: The range this highlight applies to.
    required:
      - range
    type: object
  DocumentHighlightClientCapabilities:
    additionalProperties: false
    description: Client Capabilities for a [DocumentHighlightRequest](#DocumentHighlightRequest).
    properties:
      dynamicRegistration:
        description: Whether document highlight supports dynamic registration.
        type: boolean
    type: object
  DocumentHighlightKind:
    description: A document highlight kind.
    enum:
      - 1
      - 2
      - 3
    type: number
  DocumentHighlightOptions:
    additionalProperties: false
    description: Provider options for a [DocumentHighlightRequest](#DocumentHighlightRequest).
    properties:
      workDoneProgress:
        type: boolean
    type: object
  DocumentLink:
    additionalProperties: false
    description:
      "A document link is a range in a text document that links to an internal\
      \ or external resource, like another\r\ntext document or a web site. \n The\
      \ DocumentLink namespace provides helper functions to work with\r\n[DocumentLink](#DocumentLink)\
      \ literals."
    properties:
      data:
        description:
          "A data entry field that is preserved on a document link between\
          \ a\r\nDocumentLinkRequest and a DocumentLinkResolveRequest."
      range:
        $ref: "#/definitions/Range"
        description: The range this link applies to.
      target:
        description: The uri this link points to.
        type: string
      tooltip:
        description:
          "The tooltip text when you hover over this link.\r\n\r\nIf a\
          \ tooltip is provided, is will be displayed in a string that includes instructions\
          \ on how to\r\ntrigger the link, such as `{0} (ctrl + click)`. The specific\
          \ instructions vary depending on OS,\r\nuser settings, and localization."
        type: string
    required:
      - range
    type: object
  DocumentLinkClientCapabilities:
    additionalProperties: false
    description: The client capabilities of a [DocumentLinkRequest](#DocumentLinkRequest).
    properties:
      dynamicRegistration:
        description: Whether document link supports dynamic registration.
        type: boolean
      tooltipSupport:
        description: Whether the client support the `tooltip` property on `DocumentLink`.
        type: boolean
    type: object
  DocumentLinkOptions:
    additionalProperties: false
    description: Provider options for a [DocumentLinkRequest](#DocumentLinkRequest).
    properties:
      resolveProvider:
        description: Document links have a resolve provider as well.
        type: boolean
      workDoneProgress:
        type: boolean
    type: object
  DocumentLinkParams:
    additionalProperties: false
    description: The parameters of a [DocumentLinkRequest](#DocumentLinkRequest).
    properties:
      partialResultToken:
        $ref: "#/definitions/ProgressToken"
        description:
          "An optional token that a server can use to report partial results
          (e.g. streaming) to

          the client."
      textDocument:
        $ref: "#/definitions/TextDocumentIdentifier"
        description: The document to provide document links for.
      workDoneToken:
        $ref: "#/definitions/ProgressToken"
        description: An optional token that a server can use to report work done progress.
    required:
      - textDocument
    type: object
  DocumentOnTypeFormattingClientCapabilities:
    additionalProperties: false
    description: Client capabilities of a [DocumentOnTypeFormattingRequest](#DocumentOnTypeFormattingRequest).
    properties:
      dynamicRegistration:
        description: Whether on type formatting supports dynamic registration.
        type: boolean
    type: object
  DocumentOnTypeFormattingOptions:
    additionalProperties: false
    description: Provider options for a [DocumentOnTypeFormattingRequest](#DocumentOnTypeFormattingRequest).
    properties:
      firstTriggerCharacter:
        description: A character on which formatting should be triggered, like `}`.
        type: string
      moreTriggerCharacter:
        description: More trigger characters.
        items:
          type: string
        type: array
    required:
      - firstTriggerCharacter
    type: object
  DocumentOnTypeFormattingParams:
    additionalProperties: false
    description: The parameters of a [DocumentOnTypeFormattingRequest](#DocumentOnTypeFormattingRequest).
    properties:
      ch:
        description: The character that has been typed.
        type: string
      options:
        $ref: "#/definitions/FormattingOptions"
        description: The format options.
      position:
        $ref: "#/definitions/Position"
        description: The position at which this request was send.
      textDocument:
        $ref: "#/definitions/TextDocumentIdentifier"
        description: The document to format.
    required:
      - textDocument
      - position
      - ch
      - options
    type: object
  DocumentRangeFormattingClientCapabilities:
    additionalProperties: false
    description: Client capabilities of a [DocumentRangeFormattingRequest](#DocumentRangeFormattingRequest).
    properties:
      dynamicRegistration:
        description: Whether range formatting supports dynamic registration.
        type: boolean
    type: object
  DocumentRangeFormattingOptions:
    additionalProperties: false
    description: Provider options for a [DocumentRangeFormattingRequest](#DocumentRangeFormattingRequest).
    properties:
      workDoneProgress:
        type: boolean
    type: object
  DocumentRangeFormattingParams:
    additionalProperties: false
    description: The parameters of a [DocumentRangeFormattingRequest](#DocumentRangeFormattingRequest).
    properties:
      options:
        $ref: "#/definitions/FormattingOptions"
        description: The format options
      range:
        $ref: "#/definitions/Range"
        description: The range to format
      textDocument:
        $ref: "#/definitions/TextDocumentIdentifier"
        description: The document to format.
      workDoneToken:
        $ref: "#/definitions/ProgressToken"
        description: An optional token that a server can use to report work done progress.
    required:
      - textDocument
      - range
      - options
    type: object
  DocumentSelector:
    description:
      "A document selector is the combination of one or many document filters.\
      \ \n The DocumentSelector namespace provides helper functions to work with\n\
      [DocumentSelector](#DocumentSelector)s."
    items:
      anyOf:
        - type: string
        - $ref: "#/definitions/DocumentFilter"
    type: array
  DocumentSymbol:
    additionalProperties: false
    description:
      "Represents programming constructs like variables, classes, interfaces\
      \ etc.\r\nthat appear in a document. Document symbols can be hierarchical and\
      \ they\r\nhave two ranges: one that encloses its definition and one that points\
      \ to\r\nits most interesting range, e.g. the range of an identifier."
    properties:
      children:
        description: Children of this symbol, e.g. properties of a class.
        items:
          $ref: "#/definitions/DocumentSymbol"
        type: array
      deprecated:
        description: Indicates if this symbol is deprecated.
        type: boolean
      detail:
        description: More detail for this symbol, e.g the signature of a function.
        type: string
      kind:
        $ref: "#/definitions/SymbolKind"
        description: The kind of this symbol.
      name:
        description:
          "The name of this symbol. Will be displayed in the user interface\
          \ and therefore must not be\r\nan empty string or a string only consisting\
          \ of white spaces."
        type: string
      range:
        $ref: "#/definitions/Range"
        description:
          "The range enclosing this symbol not including leading/trailing\
          \ whitespace but everything else\r\nlike comments. This information is typically\
          \ used to determine if the the clients cursor is\r\ninside the symbol to\
          \ reveal in the symbol in the UI."
      selectionRange:
        $ref: "#/definitions/Range"
        description:
          "The range that should be selected and revealed when this symbol\
          \ is being picked, e.g the name of a function.\r\nMust be contained by the\
          \ the `range`."
    required:
      - name
      - kind
      - range
      - selectionRange
    type: object
  DocumentSymbolClientCapabilities:
    additionalProperties: false
    description: Client Capabilities for a [DocumentSymbolRequest](#DocumentSymbolRequest).
    properties:
      dynamicRegistration:
        description: Whether document symbol supports dynamic registration.
        type: boolean
      hierarchicalDocumentSymbolSupport:
        description: The client support hierarchical document symbols.
        type: boolean
      symbolKind:
        additionalProperties: false
        description: Specific capabilities for the `SymbolKind`.
        properties:
          valueSet:
            description: "The symbol kind values the client supports. When this

              property exists the client also guarantees that it will

              handle values outside its set gracefully and falls back

              to a default value when unknown.


              If this property is not present the client only supports

              the symbol kinds from `File` to `Array` as defined in

              the initial version of the protocol."
            items:
              $ref: "#/definitions/SymbolKind"
            type: array
        type: object
    type: object
  DocumentSymbolOptions:
    additionalProperties: false
    description: Provider options for a [DocumentSymbolRequest](#DocumentSymbolRequest).
    properties:
      workDoneProgress:
        type: boolean
    type: object
  DocumentSymbolParams:
    additionalProperties: false
    description: Parameters for a [DocumentSymbolRequest](#DocumentSymbolRequest).
    properties:
      partialResultToken:
        $ref: "#/definitions/ProgressToken"
        description:
          "An optional token that a server can use to report partial results
          (e.g. streaming) to

          the client."
      textDocument:
        $ref: "#/definitions/TextDocumentIdentifier"
        description: The text document.
      workDoneToken:
        $ref: "#/definitions/ProgressToken"
        description: An optional token that a server can use to report work done progress.
    required:
      - textDocument
    type: object
  DocumentUri:
    description: A tagging type for string properties that are actually URIs.
    type: string
  ExecuteCommandClientCapabilities:
    additionalProperties: false
    description: The client capabilities of a [ExecuteCommandRequest](#ExecuteCommandRequest).
    properties:
      dynamicRegistration:
        description: Execute command supports dynamic registration.
        type: boolean
    type: object
  ExecuteCommandOptions:
    additionalProperties: false
    description: The server capabilities of a [ExecuteCommandRequest](#ExecuteCommandRequest).
    properties:
      commands:
        description: The commands to be executed on the server
        items:
          type: string
        type: array
      workDoneProgress:
        type: boolean
    required:
      - commands
    type: object
  ExecuteCommandParams:
    additionalProperties: false
    description: The parameters of a [ExecuteCommandRequest](#ExecuteCommandRequest).
    properties:
      arguments:
        description: Arguments that the command should be invoked with.
        items: {}
        type: array
      command:
        description: The identifier of the actual command handler.
        type: string
      workDoneToken:
        $ref: "#/definitions/ProgressToken"
        description: An optional token that a server can use to report work done progress.
    required:
      - command
    type: object
  FailureHandlingKind:
    enum:
      - abort
      - transactional
      - undo
      - textOnlyTransactional
    type: string
  FileChangeType:
    description: The file event type
    enum:
      - 1
      - 2
      - 3
    type: number
  FileEvent:
    additionalProperties: false
    description: An event describing a file change.
    properties:
      type:
        $ref: "#/definitions/FileChangeType"
        description: The change type.
      uri:
        $ref: "#/definitions/DocumentUri"
        description: The file's uri.
    required:
      - uri
      - type
    type: object
  FoldingRange:
    additionalProperties: false
    description:
      "Represents a folding range. \n The folding range namespace provides\
      \ helper functions to work with\r\n[FoldingRange](#FoldingRange) literals."
    properties:
      endCharacter:
        description:
          The zero-based character offset before the folded range ends.
          If not defined, defaults to the length of the end line.
        type: number
      endLine:
        description: The zero-based line number where the folded range ends.
        type: number
      kind:
        description:
          "Describes the kind of the folding range such as `comment' or\
          \ 'region'. The kind\r\nis used to categorize folding ranges and used by\
          \ commands like 'Fold all comments'. See\r\n[FoldingRangeKind](#FoldingRangeKind)\
          \ for an enumeration of standardized kinds."
        type: string
      startCharacter:
        description:
          The zero-based character offset from where the folded range starts.
          If not defined, defaults to the length of the start line.
        type: number
      startLine:
        description: The zero-based line number from where the folded range starts.
        type: number
    required:
      - startLine
      - endLine
    type: object
  FoldingRangeClientCapabilities:
    additionalProperties: false
    properties:
      dynamicRegistration:
        description:
          "Whether implementation supports dynamic registration for folding
          range providers. If this is set to `true`

          the client supports the new `FoldingRangeRegistrationOptions` return value
          for the corresponding server

          capability as well."
        type: boolean
      lineFoldingOnly:
        description:
          "If set, the client signals that it only supports folding complete
          lines. If set, client will

          ignore specified `startCharacter` and `endCharacter` properties in a FoldingRange."
        type: boolean
      rangeLimit:
        description:
          "The maximum number of folding ranges that the client prefers
          to receive per document. The value serves as a

          hint, servers are free to follow the limit."
        type: number
    type: object
  FoldingRangeOptions:
    additionalProperties: false
    properties:
      workDoneProgress:
        type: boolean
    type: object
  FoldingRangeParams:
    additionalProperties: false
    description: Parameters for a [FoldingRangeRequest](#FoldingRangeRequest).
    properties:
      partialResultToken:
        $ref: "#/definitions/ProgressToken"
        description:
          "An optional token that a server can use to report partial results
          (e.g. streaming) to

          the client."
      textDocument:
        $ref: "#/definitions/TextDocumentIdentifier"
        description: The text document.
      workDoneToken:
        $ref: "#/definitions/ProgressToken"
        description: An optional token that a server can use to report work done progress.
    required:
      - textDocument
    type: object
  FoldingRangeRegistrationOptions:
    additionalProperties: false
    properties:
      documentSelector:
        anyOf:
          - $ref: "#/definitions/DocumentSelector"
          - type: "null"
        description:
          "A document selector to identify the scope of the registration.
          If set to null

          the document selector provided on the client side will be used."
      id:
        description:
          "The id used to register the request. The id can be used to deregister

          the request again. See also Registration#id."
        type: string
      workDoneProgress:
        type: boolean
    required:
      - documentSelector
    type: object
  FormattingOptions:
    additionalProperties:
      anyOf:
        - type: boolean
        - type: number
        - type: string
        - not: {}
      description: Signature for further properties.
    description:
      "Value-object describing what options formatting should use. \n The\
      \ FormattingOptions namespace provides helper functions to work with\r\n[FormattingOptions](#FormattingOptions)\
      \ literals."
    properties:
      insertFinalNewline:
        description:
          Insert a newline character at the end of the file if one does
          not exist.
        type: boolean
      insertSpaces:
        description: Prefer spaces over tabs.
        type: boolean
      tabSize:
        description: Size of a tab in spaces.
        type: number
      trimFinalNewlines:
        description: Trim all newlines after the final newline at the end of the file.
        type: boolean
      trimTrailingWhitespace:
        description: Trim trailing whitespaces on a line.
        type: boolean
    required:
      - tabSize
      - insertSpaces
    type: object
  Hover:
    additionalProperties: false
    description: The result of a hover request.
    properties:
      contents:
        anyOf:
          - $ref: "#/definitions/MarkupContent"
          - $ref: "#/definitions/MarkedString"
          - items:
              $ref: "#/definitions/MarkedString"
            type: array
        description: The hover's content
      range:
        $ref: "#/definitions/Range"
        description: An optional range
    required:
      - contents
    type: object
  HoverClientCapabilities:
    additionalProperties: false
    properties:
      contentFormat:
        description: "Client supports the follow content formats for the content

          property. The order describes the preferred format of the client."
        items:
          $ref: "#/definitions/MarkupKind"
        type: array
      dynamicRegistration:
        description: Whether hover supports dynamic registration.
        type: boolean
    type: object
  HoverOptions:
    additionalProperties: false
    description: Hover options.
    properties:
      workDoneProgress:
        type: boolean
    type: object
  ImplementationClientCapabilities:
    additionalProperties: false
    description: Since 3.6.0
    properties:
      dynamicRegistration:
        description:
          "Whether implementation supports dynamic registration. If this
          is set to `true`

          the client supports the new `ImplementationRegistrationOptions` return value

          for the corresponding server capability as well."
        type: boolean
      linkSupport:
        description:
          "The client supports additional metadata in the form of definition
          links.


          Since 3.14.0"
        type: boolean
    type: object
  ImplementationOptions:
    additionalProperties: false
    properties:
      workDoneProgress:
        type: boolean
    type: object
  ImplementationRegistrationOptions:
    additionalProperties: false
    properties:
      documentSelector:
        anyOf:
          - $ref: "#/definitions/DocumentSelector"
          - type: "null"
        description:
          "A document selector to identify the scope of the registration.
          If set to null

          the document selector provided on the client side will be used."
      id:
        description:
          "The id used to register the request. The id can be used to deregister

          the request again. See also Registration#id."
        type: string
      workDoneProgress:
        type: boolean
    required:
      - documentSelector
    type: object
  InitializeParams:
    additionalProperties: false
    properties:
      capabilities:
        $ref: "#/definitions/ClientCapabilities"
        description: The capabilities provided by the client (editor or tool)
      clientInfo:
        additionalProperties: false
        description: Information about the client
        properties:
          name:
            description: The name of the client as defined by the client.
            type: string
          version:
            description: The client's version as defined by the client.
            type: string
        required:
          - name
        type: object
      initializationOptions:
        description: User provided initialization options.
      processId:
        description: "The process Id of the parent process that started

          the server."
        type:
          - number
          - "null"
      rootPath:
        description: "The rootPath of the workspace. Is null

          if no folder is open."
        type:
          - string
          - "null"
      rootUri:
        anyOf:
          - $ref: "#/definitions/DocumentUri"
          - type: "null"
        description: "The rootUri of the workspace. Is null if no

          folder is open. If both `rootPath` and `rootUri` are set

          `rootUri` wins."
      trace:
        description: The initial trace setting. If omitted trace is disabled ('off').
        enum:
          - "off"
          - messages
          - verbose
        type: string
      workDoneToken:
        $ref: "#/definitions/ProgressToken"
        description: An optional token that a server can use to report work done progress.
      workspaceFolders:
        anyOf:
          - items:
              $ref: "#/definitions/WorkspaceFolder"
            type: array
          - type: "null"
        description: The actual configured workspace folders.
    required:
      - capabilities
      - processId
      - rootUri
      - workspaceFolders
    type: object
  InitializeResult:
    additionalProperties:
      description: Custom initialization results.
    description: The result returned from an initialize request.
    properties:
      capabilities:
        $ref: "#/definitions/ServerCapabilities<any>"
        description: The capabilities the language server provides.
      serverInfo:
        additionalProperties: false
        description: Information about the server.
        properties:
          name:
            description: The name of the server as defined by the server.
            type: string
          version:
            description: The servers's version as defined by the server.
            type: string
        required:
          - name
        type: object
    required:
      - capabilities
    type: object
  InitializedParams:
    additionalProperties: false
    type: object
  InsertTextFormat:
    description:
      "Defines whether the insert text in a completion item should be interpreted\
      \ as\r\nplain text or a snippet."
    enum:
      - 1
      - 2
    type: number
  Location:
    additionalProperties: false
    description:
      "Represents a location inside a resource, such as a line\r\ninside\
      \ a text file. \n The Location namespace provides helper functions to work with\r\
      \n[Location](#Location) literals."
    properties:
      range:
        $ref: "#/definitions/Range"
      uri:
        $ref: "#/definitions/DocumentUri"
    required:
      - uri
      - range
    type: object
  LogMessageParams:
    additionalProperties: false
    description: The log message parameters.
    properties:
      message:
        description: The actual message
        type: string
      type:
        $ref: "#/definitions/MessageType"
        description: The message type. See {@link MessageType}
    required:
      - type
      - message
    type: object
  MarkedString:
    anyOf:
      - type: string
      - additionalProperties: false
        properties:
          language:
            type: string
          value:
            type: string
        required:
          - language
          - value
        type: object
    description:
      "MarkedString can be used to render human readable text. It is either\
      \ a markdown string\r\nor a code-block that provides a language and a code snippet.\
      \ The language identifier\r\nis semantically equal to the optional language\
      \ identifier in fenced code blocks in GitHub\r\nissues. See https://help.github.com/articles/creating-and-highlighting-code-blocks/#syntax-highlighting\r\
      \n\r\nThe pair of a language and a value is an equivalent to markdown:\r\n```${language}\r\
      \n${value}\r\n```\r\n\r\nNote that markdown strings will be sanitized - that\
      \ means html will be escaped."
  MarkupContent:
    additionalProperties: false
    description:
      "A `MarkupContent` literal represents a string value which content\
      \ is interpreted base on its\r\nkind flag. Currently the protocol supports `plaintext`\
      \ and `markdown` as markup kinds.\r\n\r\nIf the kind is `markdown` then the\
      \ value can contain fenced code blocks like in GitHub issues.\r\nSee https://help.github.com/articles/creating-and-highlighting-code-blocks/#syntax-highlighting\r\
      \n\r\nHere is an example how such a string can be constructed using JavaScript\
      \ / TypeScript:\r\n```ts\r\nlet markdown: MarkdownContent = {\r\n  kind: MarkupKind.Markdown,\r\
      \nvalue: [\r\n\t\t'# Header',\r\n\t\t'Some text',\r\n\t\t'```typescript',\r\n\
      \t\t'someCode();',\r\n\t\t'```'\r\n].join('\\n')\r\n};\r\n```\r\n\r\n*Please\
      \ Note* that clients might sanitize the return markdown. A client could decide\
      \ to\r\nremove HTML from the markdown to avoid script execution."
    properties:
      kind:
        $ref: "#/definitions/MarkupKind"
        description: The type of the Markup
      value:
        description: The content itself
        type: string
    required:
      - kind
      - value
    type: object
  MarkupKind:
    description:
      "Describes the content type that a client supports in various\r\n\
      result literals like `Hover`, `ParameterInfo` or `CompletionItem`.\r\n\r\nPlease\
      \ note that `MarkupKinds` must not start with a `$`. This kinds\r\nare reserved\
      \ for internal usage."
    enum:
      - plaintext
      - markdown
    type: string
  MessageActionItem:
    additionalProperties: false
    properties:
      title:
        description: A short title like 'Retry', 'Open Log' etc.
        type: string
    required:
      - title
    type: object
  MessageType:
    description: The message type
    enum:
      - 1
      - 2
      - 3
      - 4
    type: number
  ParameterInformation:
    additionalProperties: false
    description:
      "Represents a parameter of a callable-signature. A parameter can\r\
      \nhave a label and a doc-comment. \n The ParameterInformation namespace provides\
      \ helper functions to work with\r\n[ParameterInformation](#ParameterInformation)\
      \ literals."
    properties:
      documentation:
        anyOf:
          - type: string
          - $ref: "#/definitions/MarkupContent"
        description:
          "The human-readable doc-comment of this signature. Will be shown\r\
          \nin the UI but can be omitted."
      label:
        anyOf:
          - type: string
          - items:
              - type: number
              - type: number
            maxItems: 2
            minItems: 2
            type: array
        description:
          "The label of this parameter information.\r\n\r\nEither a string\
          \ or an inclusive start and exclusive end offsets within its containing\r\
          \nsignature label. (see SignatureInformation.label). The offsets are based\
          \ on a UTF-16\r\nstring representation as `Position` and `Range` does.\r\
          \n\r\n*Note*: a label of type string should be a substring of its containing\
          \ signature label.\r\nIts intended use case is to highlight the parameter\
          \ label part in the `SignatureInformation.label`."
    required:
      - label
    type: object
  Position:
    additionalProperties: false
    description:
      "Position in a text document expressed as zero-based line and character\
      \ offset.\r\nThe offsets are based on a UTF-16 string representation. So a string\
      \ of the form\r\n`a\U00010400b` the character offset of the character `a` is\
      \ 0, the character offset of `\U00010400`\r\nis 1 and the character offset of\
      \ b is 3 since `\U00010400` is represented using two code\r\nunits in UTF-16.\r\
      \n\r\nPositions are line end character agnostic. So you can not specify a position\
      \ that\r\ndenotes `\\r|\\n` or `\\n|` where `|` represents the character offset.\
      \ \n The Position namespace provides helper functions to work with\r\n[Position](#Position)\
      \ literals."
    properties:
      character:
        description:
          "Character offset on a line in a document (zero-based). Assuming\
          \ that the line is\r\nrepresented as a string, the `character` value represents\
          \ the gap between the\r\n`character` and `character + 1`.\r\n\r\nIf the\
          \ character value is greater than the line length it defaults back to the\r\
          \nline length.\r\nIf a line number is negative, it defaults to 0."
        type: number
      line:
        description:
          "Line position in a document (zero-based).\r\nIf a line number\
          \ is greater than the number of lines in a document, it defaults back to\
          \ the number of lines in the document.\r\nIf a line number is negative,\
          \ it defaults to 0."
        type: number
    required:
      - line
      - character
    type: object
  ProgressToken:
    type:
      - number
      - string
  PublishDiagnosticsClientCapabilities:
    additionalProperties: false
    description: The publish diagnostic client capabilities.
    properties:
      relatedInformation:
        description: Whether the clients accepts diagnostics with related information.
        type: boolean
      tagSupport:
        additionalProperties: false
        description:
          "Client supports the tag property to provide meta data about
          a diagnostic.

          Clients supporting tags have to handle unknown tags gracefully."
        properties:
          valueSet:
            description: The tags supported by the client.
            items:
              $ref: "#/definitions/DiagnosticTag"
            type: array
        required:
          - valueSet
        type: object
    type: object
  PublishDiagnosticsParams:
    additionalProperties: false
    description: The publish diagnostic notification's parameters.
    properties:
      diagnostics:
        description: An array of diagnostic information items.
        items:
          $ref: "#/definitions/Diagnostic"
        type: array
      uri:
        $ref: "#/definitions/DocumentUri"
        description: The URI for which diagnostic information is reported.
      version:
        description:
          Optional the version number of the document the diagnostics are
          published for.
        type: number
    required:
      - uri
      - diagnostics
    type: object
  Range:
    additionalProperties: false
    description:
      "A range in a text document expressed as (zero-based) start and end\
      \ positions.\r\n\r\nIf you want to specify a range that contains a line including\
      \ the line ending\r\ncharacter(s) then use an end position denoting the start\
      \ of the next line.\r\nFor example:\r\n```ts\r\n{\r\n     start: { line: 5,\
      \ character: 23 }\r\n     end : { line 6, character : 0 }\r\n}\r\n``` \n The\
      \ Range namespace provides helper functions to work with\r\n[Range](#Range)\
      \ literals."
    properties:
      end:
        $ref: "#/definitions/Position"
        description: The range's end position.
      start:
        $ref: "#/definitions/Position"
        description: The range's start position
    required:
      - start
      - end
    type: object
  ReferenceClientCapabilities:
    additionalProperties: false
    description: Client Capabilities for a [ReferencesRequest](#ReferencesRequest).
    properties:
      dynamicRegistration:
        description: Whether references supports dynamic registration.
        type: boolean
    type: object
  ReferenceContext:
    additionalProperties: false
    description:
      "Value-object that contains additional information when\r\nrequesting\
      \ references."
    properties:
      includeDeclaration:
        description: Include the declaration of the current symbol.
        type: boolean
    required:
      - includeDeclaration
    type: object
  ReferenceOptions:
    additionalProperties: false
    description: Reference options.
    properties:
      workDoneProgress:
        type: boolean
    type: object
  ReferenceParams:
    additionalProperties: false
    description: Parameters for a [ReferencesRequest](#ReferencesRequest).
    properties:
      context:
        $ref: "#/definitions/ReferenceContext"
      partialResultToken:
        $ref: "#/definitions/ProgressToken"
        description:
          "An optional token that a server can use to report partial results
          (e.g. streaming) to

          the client."
      position:
        $ref: "#/definitions/Position"
        description: The position inside the text document.
      textDocument:
        $ref: "#/definitions/TextDocumentIdentifier"
        description: The text document.
      workDoneToken:
        $ref: "#/definitions/ProgressToken"
        description: An optional token that a server can use to report work done progress.
    required:
      - context
      - position
      - textDocument
    type: object
  Registration:
    additionalProperties: false
    description:
      General parameters to to register for an notification or to register
      a provider.
    properties:
      id:
        description:
          "The id used to register the request. The id can be used to deregister

          the request again."
        type: string
      method:
        description: The method to register for.
        type: string
      registerOptions:
        description: Options necessary for the registration.
    required:
      - id
      - method
    type: object
  RegistrationParams:
    additionalProperties: false
    properties:
      registrations:
        items:
          $ref: "#/definitions/Registration"
        type: array
    required:
      - registrations
    type: object
  RenameClientCapabilities:
    additionalProperties: false
    properties:
      dynamicRegistration:
        description: Whether rename supports dynamic registration.
        type: boolean
      prepareSupport:
        description: "Client supports testing for validity of rename operations

          before execution."
        type: boolean
    type: object
  RenameFile:
    additionalProperties: false
    description: Rename file operation
    properties:
      kind:
        description: A rename
        enum:
          - rename
        type: string
      newUri:
        $ref: "#/definitions/DocumentUri"
        description: The new location.
      oldUri:
        $ref: "#/definitions/DocumentUri"
        description: The old (existing) location.
      options:
        $ref: "#/definitions/RenameFileOptions"
        description: Rename options.
    required:
      - kind
      - newUri
      - oldUri
    type: object
  RenameFileOptions:
    additionalProperties: false
    description: Rename file options
    properties:
      ignoreIfExists:
        description: Ignores if target exists.
        type: boolean
      overwrite:
        description: Overwrite target if existing. Overwrite wins over `ignoreIfExists`
        type: boolean
    type: object
  RenameOptions:
    additionalProperties: false
    description: Provider options for a [RenameRequest](#RenameRequest).
    properties:
      prepareProvider:
        description: Renames should be checked and tested before being executed.
        type: boolean
      workDoneProgress:
        type: boolean
    type: object
  RenameParams:
    additionalProperties: false
    description: The parameters of a [RenameRequest](#RenameRequest).
    properties:
      newName:
        description:
          "The new name of the symbol. If the given name is not valid the

          request must return a [ResponseError](#ResponseError) with an

          appropriate message set."
        type: string
      position:
        $ref: "#/definitions/Position"
        description: The position at which this request was sent.
      textDocument:
        $ref: "#/definitions/TextDocumentIdentifier"
        description: The document to rename.
      workDoneToken:
        $ref: "#/definitions/ProgressToken"
        description: An optional token that a server can use to report work done progress.
    required:
      - textDocument
      - position
      - newName
    type: object
  ResourceOperationKind:
    description: The kind of resource operations supported by the client.
    enum:
      - create
      - rename
      - delete
    type: string
  ResponseErrorLiteral<any>:
    additionalProperties: false
    properties:
      code:
        description: A number indicating the error type that occured.
        type: number
      data:
        description:
          "A Primitive or Structured value that contains additional\r\n\
          information about the error. Can be omitted."
      message:
        description: A string providing a short decription of the error.
        type: string
    required:
      - code
      - message
    type: object
  SaveOptions:
    additionalProperties: false
    description: Save options.
    properties:
      includeText:
        description: The client is supposed to include the content on save.
        type: boolean
    type: object
  SelectionRangeClientCapabilities:
    additionalProperties: false
    properties:
      dynamicRegistration:
        description:
          "Whether implementation supports dynamic registration for selection
          range providers. If this is set to `true`

          the client supports the new `SelectionRangeRegistrationOptions` return value
          for the corresponding server

          capability as well."
        type: boolean
    type: object
  SelectionRangeOptions:
    additionalProperties: false
    properties:
      workDoneProgress:
        type: boolean
    type: object
  SelectionRangeRegistrationOptions:
    additionalProperties: false
    properties:
      documentSelector:
        anyOf:
          - $ref: "#/definitions/DocumentSelector"
          - type: "null"
        description:
          "A document selector to identify the scope of the registration.
          If set to null

          the document selector provided on the client side will be used."
      id:
        description:
          "The id used to register the request. The id can be used to deregister

          the request again. See also Registration#id."
        type: string
      workDoneProgress:
        type: boolean
    required:
      - documentSelector
    type: object
  ServerCapabilities<any>:
    additionalProperties: false
    properties:
      codeActionProvider:
        anyOf:
          - type: boolean
          - $ref: "#/definitions/CodeActionOptions"
        description:
          "The server provides code actions. CodeActionOptions may only
          be

          specified if the client states that it supports

          `codeActionLiteralSupport` in its initial `initialize` request."
      codeLensProvider:
        $ref: "#/definitions/CodeLensOptions"
        description: The server provides code lens.
      colorProvider:
        anyOf:
          - type: boolean
          - $ref: "#/definitions/DocumentColorOptions"
          - $ref: "#/definitions/DocumentColorRegistrationOptions"
        description: The server provides color provider support.
      completionProvider:
        $ref: "#/definitions/CompletionOptions"
        description: The server provides completion support.
      declarationProvider:
        anyOf:
          - type: boolean
          - $ref: "#/definitions/DeclarationOptions"
          - $ref: "#/definitions/DeclarationRegistrationOptions"
        description: The server provides Goto Declaration support.
      definitionProvider:
        anyOf:
          - type: boolean
          - $ref: "#/definitions/DefinitionOptions"
        description: The server provides goto definition support.
      documentFormattingProvider:
        anyOf:
          - type: boolean
          - $ref: "#/definitions/DocumentFormattingOptions"
        description: The server provides document formatting.
      documentHighlightProvider:
        anyOf:
          - type: boolean
          - $ref: "#/definitions/DocumentHighlightOptions"
        description: The server provides document highlight support.
      documentLinkProvider:
        $ref: "#/definitions/DocumentLinkOptions"
        description: The server provides document link support.
      documentOnTypeFormattingProvider:
        $ref: "#/definitions/DocumentOnTypeFormattingOptions"
        description: The server provides document formatting on typing.
      documentRangeFormattingProvider:
        anyOf:
          - type: boolean
          - $ref: "#/definitions/DocumentRangeFormattingOptions"
        description: The server provides document range formatting.
      documentSymbolProvider:
        anyOf:
          - type: boolean
          - $ref: "#/definitions/DocumentSymbolOptions"
        description: The server provides document symbol support.
      executeCommandProvider:
        $ref: "#/definitions/ExecuteCommandOptions"
        description: The server provides execute command support.
      experimental:
        description: Experimental server capabilities.
      foldingRangeProvider:
        anyOf:
          - type: boolean
          - $ref: "#/definitions/FoldingRangeOptions"
          - $ref: "#/definitions/FoldingRangeRegistrationOptions"
        description: The server provides folding provider support.
      hoverProvider:
        anyOf:
          - type: boolean
          - $ref: "#/definitions/HoverOptions"
        description: The server provides hover support.
      implementationProvider:
        anyOf:
          - type: boolean
          - $ref: "#/definitions/ImplementationOptions"
          - $ref: "#/definitions/ImplementationRegistrationOptions"
        description: The server provides Goto Implementation support.
      referencesProvider:
        anyOf:
          - type: boolean
          - $ref: "#/definitions/ReferenceOptions"
        description: The server provides find references support.
      renameProvider:
        anyOf:
          - type: boolean
          - $ref: "#/definitions/RenameOptions"
        description:
          "The server provides rename support. RenameOptions may only be

          specified if the client states that it supports

          `prepareSupport` in its initial `initialize` request."
      selectionRangeProvider:
        anyOf:
          - type: boolean
          - $ref: "#/definitions/SelectionRangeOptions"
          - $ref: "#/definitions/SelectionRangeRegistrationOptions"
        description: The server provides selection range support.
      signatureHelpProvider:
        $ref: "#/definitions/SignatureHelpOptions"
        description: The server provides signature help support.
      textDocumentSync:
        anyOf:
          - $ref: "#/definitions/TextDocumentSyncOptions"
          - $ref: "#/definitions/TextDocumentSyncKind"
        description:
          "Defines how text documents are synced. Is either a detailed
          structure defining each notification or

          for backwards compatibility the TextDocumentSyncKind number."
      typeDefinitionProvider:
        anyOf:
          - type: boolean
          - $ref: "#/definitions/TypeDefinitionOptions"
          - $ref: "#/definitions/TypeDefinitionRegistrationOptions"
        description: The server provides Goto Type Definition support.
      workspace:
        additionalProperties: false
        description: The workspace server capabilities
        properties:
          workspaceFolders:
            additionalProperties: false
            properties:
              changeNotifications:
                description:
                  "Whether the server wants to receive workspace folder

                  change notifications.


                  If a strings is provided the string is treated as a ID

                  under which the notification is registed on the client

                  side. The ID can be used to unregister for these events

                  using the `client/unregisterCapability` request."
                type:
                  - string
                  - boolean
              supported:
                description: The Server has support for workspace folders
                type: boolean
            type: object
        type: object
      workspaceSymbolProvider:
        anyOf:
          - type: boolean
          - $ref: "#/definitions/WorkspaceSymbolOptions"
        description: The server provides workspace symbol support.
    type: object
  ShowMessageParams:
    additionalProperties: false
    description: The parameters of a notification message.
    properties:
      message:
        description: The actual message
        type: string
      type:
        $ref: "#/definitions/MessageType"
        description: The message type. See {@link MessageType}
    required:
      - type
      - message
    type: object
  ShowMessageRequestParams:
    additionalProperties: false
    properties:
      actions:
        description: The message action items to present.
        items:
          $ref: "#/definitions/MessageActionItem"
        type: array
      message:
        description: The actual message
        type: string
      type:
        $ref: "#/definitions/MessageType"
        description: The message type. See {@link MessageType}
    required:
      - type
      - message
    type: object
  SignatureHelp:
    additionalProperties: false
    description:
      "Signature help represents the signature of something\r\ncallable.\
      \ There can be multiple signature but only one\r\nactive and only one active\
      \ parameter."
    properties:
      activeParameter:
        description:
          "The active parameter of the active signature. Set to `null`\r\
          \nif the active signature has no parameters."
        type:
          - number
          - "null"
      activeSignature:
        description: "The active signature. Set to `null` if no\r\nsignatures exist."
        type:
          - number
          - "null"
      signatures:
        description: One or more signatures.
        items:
          $ref: "#/definitions/SignatureInformation"
        type: array
    required:
      - signatures
      - activeSignature
      - activeParameter
    type: object
  SignatureHelpClientCapabilities:
    additionalProperties: false
    description: Client Capabilities for a [SignatureHelpRequest](#SignatureHelpRequest).
    properties:
      contextSupport:
        description:
          "The client supports to send additional context information for
          a

          `textDocument/signatureHelp` request. A client that opts into

          contextSupport will also support the `retriggerCharacters` on

          `SignatureHelpOptions`."
        type: boolean
      dynamicRegistration:
        description: Whether signature help supports dynamic registration.
        type: boolean
      signatureInformation:
        additionalProperties: false
        description: "The client supports the following `SignatureInformation`

          specific properties."
        properties:
          documentationFormat:
            description:
              "Client supports the follow content formats for the documentation

              property. The order describes the preferred format of the client."
            items:
              $ref: "#/definitions/MarkupKind"
            type: array
          parameterInformation:
            additionalProperties: false
            description: Client capabilities specific to parameter information.
            properties:
              labelOffsetSupport:
                description:
                  "The client supports processing label offsets instead
                  of a

                  simple label string."
                type: boolean
            type: object
        type: object
    type: object
  SignatureHelpOptions:
    additionalProperties: false
    description: Server Capabilities for a [SignatureHelpRequest](#SignatureHelpRequest).
    properties:
      retriggerCharacters:
        description: "List of characters that re-trigger signature help.


          These trigger characters are only active when signature help is already
          showing. All trigger characters

          are also counted as re-trigger characters."
        items:
          type: string
        type: array
      triggerCharacters:
        description: List of characters that trigger signature help.
        items:
          type: string
        type: array
      workDoneProgress:
        type: boolean
    type: object
  SignatureInformation:
    additionalProperties: false
    description:
      "Represents the signature of something callable. A signature\r\n\
      can have a label, like a function-name, a doc-comment, and\r\na set of parameters.\
      \ \n The SignatureInformation namespace provides helper functions to work with\r\
      \n[SignatureInformation](#SignatureInformation) literals."
    properties:
      documentation:
        anyOf:
          - type: string
          - $ref: "#/definitions/MarkupContent"
        description:
          "The human-readable doc-comment of this signature. Will be shown\r\
          \nin the UI but can be omitted."
      label:
        description: "The label of this signature. Will be shown in\r\nthe UI."
        type: string
      parameters:
        description: The parameters of this signature.
        items:
          $ref: "#/definitions/ParameterInformation"
        type: array
    required:
      - label
    type: object
  SymbolInformation:
    additionalProperties: false
    description:
      "Represents information about programming constructs like variables,\
      \ classes,\r\ninterfaces etc."
    properties:
      containerName:
        description:
          "The name of the symbol containing this symbol. This information\
          \ is for\r\nuser interface purposes (e.g. to render a qualifier in the user\
          \ interface\r\nif necessary). It can't be used to re-infer a hierarchy for\
          \ the document\r\nsymbols."
        type: string
      deprecated:
        description: Indicates if this symbol is deprecated.
        type: boolean
      kind:
        $ref: "#/definitions/SymbolKind"
        description: The kind of this symbol.
      location:
        $ref: "#/definitions/Location"
        description:
          "The location of this symbol. The location's range is used by\
          \ a tool\r\nto reveal the location in the editor. If the symbol is selected\
          \ in the\r\ntool the range's start information is used to position the cursor.\
          \ So\r\nthe range usually spans more than the actual symbol's name and does\r\
          \nnormally include thinks like visibility modifiers.\r\n\r\nThe range doesn't\
          \ have to denote a node range in the sense of a abstract\r\nsyntax tree.\
          \ It can therefore not be used to re-construct a hierarchy of\r\nthe symbols."
      name:
        description: The name of this symbol.
        type: string
    required:
      - name
      - kind
      - location
    type: object
  SymbolKind:
    description: A symbol kind.
    enum:
      - 1
      - 2
      - 3
      - 4
      - 5
      - 6
      - 7
      - 8
      - 9
      - 10
      - 11
      - 12
      - 13
      - 14
      - 15
      - 16
      - 17
      - 18
      - 19
      - 20
      - 21
      - 22
      - 23
      - 24
      - 25
      - 26
    type: number
  TextDocumentClientCapabilities:
    additionalProperties: false
    description: Text document specific client capabilities.
    properties:
      codeAction:
        $ref: "#/definitions/CodeActionClientCapabilities"
        description: Capabilities specific to the `textDocument/codeAction`
      codeLens:
        $ref: "#/definitions/CodeLensClientCapabilities"
        description: Capabilities specific to the `textDocument/codeLens`
      colorProvider:
        $ref: "#/definitions/DocumentColorClientCapabilities"
        description: Capabilities specific to the `textDocument/documentColor`
      completion:
        $ref: "#/definitions/CompletionClientCapabilities"
        description: Capabilities specific to the `textDocument/completion`
      declaration:
        $ref: "#/definitions/DeclarationClientCapabilities"
        description: Capabilities specific to the `textDocument/declaration`
      definition:
        $ref: "#/definitions/DefinitionClientCapabilities"
        description: Capabilities specific to the `textDocument/definition`
      documentHighlight:
        $ref: "#/definitions/DocumentHighlightClientCapabilities"
        description: Capabilities specific to the `textDocument/documentHighlight`
      documentLink:
        $ref: "#/definitions/DocumentLinkClientCapabilities"
        description: Capabilities specific to the `textDocument/documentLink`
      documentSymbol:
        $ref: "#/definitions/DocumentSymbolClientCapabilities"
        description: Capabilities specific to the `textDocument/documentSymbol`
      foldingRange:
        $ref: "#/definitions/FoldingRangeClientCapabilities"
        description: Capabilities specific to `textDocument/foldingRange` requests.
      formatting:
        $ref: "#/definitions/DocumentFormattingClientCapabilities"
        description: Capabilities specific to the `textDocument/formatting`
      hover:
        $ref: "#/definitions/HoverClientCapabilities"
        description: Capabilities specific to the `textDocument/hover`
      implementation:
        $ref: "#/definitions/ImplementationClientCapabilities"
        description: Capabilities specific to the `textDocument/implementation`
      onTypeFormatting:
        $ref: "#/definitions/DocumentOnTypeFormattingClientCapabilities"
        description: Capabilities specific to the `textDocument/onTypeFormatting`
      publishDiagnostics:
        $ref: "#/definitions/PublishDiagnosticsClientCapabilities"
        description: Capabilities specific to `textDocument/publishDiagnostics`.
      rangeFormatting:
        $ref: "#/definitions/DocumentRangeFormattingClientCapabilities"
        description: Capabilities specific to the `textDocument/rangeFormatting`
      references:
        $ref: "#/definitions/ReferenceClientCapabilities"
        description: Capabilities specific to the `textDocument/references`
      rename:
        $ref: "#/definitions/RenameClientCapabilities"
        description: Capabilities specific to the `textDocument/rename`
      selectionRange:
        $ref: "#/definitions/SelectionRangeClientCapabilities"
        description: Capabilities specific to `textDocument/selectionRange` requests
      signatureHelp:
        $ref: "#/definitions/SignatureHelpClientCapabilities"
        description: Capabilities specific to the `textDocument/signatureHelp`
      synchronization:
        $ref: "#/definitions/TextDocumentSyncClientCapabilities"
        description: Defines which synchronization capabilities the client supports.
      typeDefinition:
        $ref: "#/definitions/TypeDefinitionClientCapabilities"
        description: Capabilities specific to the `textDocument/typeDefinition`
    type: object
  TextDocumentContentChangeEvent:
    additionalProperties: false
    description:
      "An event describing a change to a text document. If range and rangeLength\
      \ are omitted\r\nthe new text is considered to be the full content of the document."
    properties:
      range:
        $ref: "#/definitions/Range"
        description: The range of the document that changed.
      rangeLength:
        description: The length of the range that got replaced.
        type: number
      text:
        description: The new text of the document.
        type: string
    required:
      - text
    type: object
  TextDocumentEdit:
    additionalProperties: false
    description:
      "Describes textual changes on a text document. \n The TextDocumentEdit\
      \ namespace provides helper function to create\r\nan edit that manipulates a\
      \ text document."
    properties:
      edits:
        description: The edits to be applied.
        items:
          $ref: "#/definitions/TextEdit"
        type: array
      textDocument:
        $ref: "#/definitions/VersionedTextDocumentIdentifier"
        description: The text document to change.
    required:
      - textDocument
      - edits
    type: object
  TextDocumentIdentifier:
    additionalProperties: false
    description:
      "A literal to identify a text document in the client. \n The TextDocumentIdentifier\
      \ namespace provides helper functions to work with\r\n[TextDocumentIdentifier](#TextDocumentIdentifier)\
      \ literals."
    properties:
      uri:
        $ref: "#/definitions/DocumentUri"
        description: The text document's uri.
    required:
      - uri
    type: object
  TextDocumentItem:
    additionalProperties: false
    description:
      "An item to transfer a text document from the client to the\r\nserver.\
      \ \n The TextDocumentItem namespace provides helper functions to work with\r\
      \n[TextDocumentItem](#TextDocumentItem) literals."
    properties:
      languageId:
        description: The text document's language identifier
        type: string
      text:
        description: The content of the opened text document.
        type: string
      uri:
        $ref: "#/definitions/DocumentUri"
        description: The text document's uri.
      version:
        description:
          "The version number of this document (it will increase after\
          \ each\r\nchange, including undo/redo)."
        type: number
    required:
      - uri
      - languageId
      - version
      - text
    type: object
  TextDocumentPositionParams:
    additionalProperties: false
    description:
      "A parameter literal used in requests to pass a text document and
      a position inside that

      document."
    properties:
      position:
        $ref: "#/definitions/Position"
        description: The position inside the text document.
      textDocument:
        $ref: "#/definitions/TextDocumentIdentifier"
        description: The text document.
    required:
      - textDocument
      - position
    type: object
  TextDocumentSaveReason:
    description: Represents reasons why a text document is saved.
    enum:
      - 1
      - 2
      - 3
    type: number
  TextDocumentSyncClientCapabilities:
    additionalProperties: false
    properties:
      didSave:
        description: The client supports did save notifications.
        type: boolean
      dynamicRegistration:
        description: Whether text document synchronization supports dynamic registration.
        type: boolean
      willSave:
        description: The client supports sending will save notifications.
        type: boolean
      willSaveWaitUntil:
        description: "The client supports sending a will save request and

          waits for a response providing text edits which will

          be applied to the document before it is saved."
        type: boolean
    type: object
  TextDocumentSyncKind:
    description: "Defines how the host (editor) should sync

      document changes to the language server."
    enum:
      - 0
      - 1
      - 2
    type: number
  TextDocumentSyncOptions:
    additionalProperties: false
    properties:
      change:
        $ref: "#/definitions/TextDocumentSyncKind"
        description:
          "Change notifications are sent to the server. See TextDocumentSyncKind.None,
          TextDocumentSyncKind.Full

          and TextDocumentSyncKind.Incremental. If omitted it defaults to TextDocumentSyncKind.None."
      openClose:
        description:
          "Open and close notifications are sent to the server. If omitted
          open close notification should not

          be sent."
        type: boolean
      save:
        $ref: "#/definitions/SaveOptions"
        description:
          "If present save notifications are sent to the server. If omitted
          the notification should not be

          sent."
      willSave:
        description:
          "If present will save notifications are sent to the server. If
          omitted the notification should not be

          sent."
        type: boolean
      willSaveWaitUntil:
        description:
          "If present will save wait until requests are sent to the server.
          If omitted the request should not be

          sent."
        type: boolean
    type: object
  TextEdit:
    additionalProperties: false
    description:
      "A text edit applicable to a text document. \n The TextEdit namespace\
      \ provides helper function to create replace,\r\ninsert and delete edits more\
      \ easily."
    properties:
      newText:
        description:
          "The string to be inserted. For delete operations use an\r\n\
          empty string."
        type: string
      range:
        $ref: "#/definitions/Range"
        description:
          "The range of the text document to be manipulated. To insert\r\
          \ntext into a document create a range where start === end."
    required:
      - range
      - newText
    type: object
  TypeDefinitionClientCapabilities:
    additionalProperties: false
    description: Since 3.6.0
    properties:
      dynamicRegistration:
        description:
          "Whether implementation supports dynamic registration. If this
          is set to `true`

          the client supports the new `TypeDefinitionRegistrationOptions` return value

          for the corresponding server capability as well."
        type: boolean
      linkSupport:
        description:
          "The client supports additional metadata in the form of definition
          links.


          Since 3.14.0"
        type: boolean
    type: object
  TypeDefinitionOptions:
    additionalProperties: false
    properties:
      workDoneProgress:
        type: boolean
    type: object
  TypeDefinitionRegistrationOptions:
    additionalProperties: false
    properties:
      documentSelector:
        anyOf:
          - $ref: "#/definitions/DocumentSelector"
          - type: "null"
        description:
          "A document selector to identify the scope of the registration.
          If set to null

          the document selector provided on the client side will be used."
      id:
        description:
          "The id used to register the request. The id can be used to deregister

          the request again. See also Registration#id."
        type: string
      workDoneProgress:
        type: boolean
    required:
      - documentSelector
    type: object
  Unregistration:
    additionalProperties: false
    description: General parameters to unregister a request or notification.
    properties:
      id:
        description:
          "The id used to unregister the request or notification. Usually
          an id

          provided during the register request."
        type: string
      method:
        description: The method to unregister for.
        type: string
    required:
      - id
      - method
    type: object
  UnregistrationParams:
    additionalProperties: false
    properties:
      unregisterations:
        items:
          $ref: "#/definitions/Unregistration"
        type: array
    required:
      - unregisterations
    type: object
  VersionedTextDocumentIdentifier:
    additionalProperties: false
    description:
      "An identifier to denote a specific version of a text document. \n\
      \ The VersionedTextDocumentIdentifier namespace provides helper functions to\
      \ work with\r\n[VersionedTextDocumentIdentifier](#VersionedTextDocumentIdentifier)\
      \ literals."
    properties:
      uri:
        $ref: "#/definitions/DocumentUri"
        description: The text document's uri.
      version:
        description:
          "The version number of this document. If a versioned text document\
          \ identifier\r\nis sent from the server to the client and the file is not\
          \ open in the editor\r\n(the server has not received an open notification\
          \ before) the server can send\r\n`null` to indicate that the version is\
          \ unknown and the content on disk is the\r\ntruth (as speced with document\
          \ content ownership)."
        type:
          - number
          - "null"
    required:
      - uri
      - version
    type: object
  WillSaveTextDocumentParams:
    additionalProperties: false
    description: The parameters send in a will save text document notification.
    properties:
      reason:
        $ref: "#/definitions/TextDocumentSaveReason"
        description: The 'TextDocumentSaveReason'.
      textDocument:
        $ref: "#/definitions/TextDocumentIdentifier"
        description: The document that will be saved.
    required:
      - textDocument
      - reason
    type: object
  WorkspaceClientCapabilities:
    additionalProperties: false
    description: Workspace specific client capabilities.
    properties:
      applyEdit:
        description: "The client supports applying batch edits

          to the workspace by supporting the request

          'workspace/applyEdit'"
        type: boolean
      didChangeConfiguration:
        $ref: "#/definitions/DidChangeConfigurationClientCapabilities"
        description:
          Capabilities specific to the `workspace/didChangeConfiguration`
          notification.
      didChangeWatchedFiles:
        $ref: "#/definitions/DidChangeWatchedFilesClientCapabilities"
        description:
          Capabilities specific to the `workspace/didChangeWatchedFiles`
          notification.
      executeCommand:
        $ref: "#/definitions/ExecuteCommandClientCapabilities"
        description: Capabilities specific to the `workspace/executeCommand` request.
      symbol:
        $ref: "#/definitions/WorkspaceSymbolClientCapabilities"
        description: Capabilities specific to the `workspace/symbol` request.
      workspaceEdit:
        $ref: "#/definitions/WorkspaceEditClientCapabilities"
        description: Capabilities specific to `WorkspaceEdit`s
    type: object
  WorkspaceEdit:
    additionalProperties: false
    description:
      "A workspace edit represents changes to many resources managed in\
      \ the workspace. The edit\r\nshould either provide `changes` or `documentChanges`.\
      \ If documentChanges are present\r\nthey are preferred over `changes` if the\
      \ client can handle versioned document edits."
    properties:
      changes:
        additionalProperties:
          items:
            $ref: "#/definitions/TextEdit"
          type: array
        description: Holds changes to existing resources.
        type: object
      documentChanges:
        description:
          "Depending on the client capability `workspace.workspaceEdit.resourceOperations`\
          \ document changes\r\nare either an array of `TextDocumentEdit`s to express\
          \ changes to n different text documents\r\nwhere each text document edit\
          \ addresses a specific version of a text document. Or it can contain\r\n\
          above `TextDocumentEdit`s mixed with create, rename and delete file / folder\
          \ operations.\r\n\r\nWhether a client supports versioned document edits\
          \ is expressed via\r\n`workspace.workspaceEdit.documentChanges` client capability.\r\
          \n\r\nIf a client neither supports `documentChanges` nor `workspace.workspaceEdit.resourceOperations`\
          \ then\r\nonly plain `TextEdit`s using the `changes` property are supported."
        items:
          anyOf:
            - $ref: "#/definitions/TextDocumentEdit"
            - $ref: "#/definitions/CreateFile"
            - $ref: "#/definitions/RenameFile"
            - $ref: "#/definitions/DeleteFile"
        type: array
    type: object
  WorkspaceEditClientCapabilities:
    additionalProperties: false
    properties:
      documentChanges:
        description: The client supports versioned document changes in `WorkspaceEdit`s
        type: boolean
      failureHandling:
        $ref: "#/definitions/FailureHandlingKind"
        description:
          "The failure handling strategy of a client if applying the workspace
          edit

          fails."
      resourceOperations:
        description:
          "The resource operations the client supports. Clients should
          at least

          support 'create', 'rename' and 'delete' files and folders."
        items:
          $ref: "#/definitions/ResourceOperationKind"
        type: array
    type: object
  WorkspaceFolder:
    additionalProperties: false
    properties:
      name:
        description: "The name of the workspace folder. Used to refer to this

          workspace folder in thge user interface."
        type: string
      uri:
        description: The associated URI for this workspace folder.
        type: string
    required:
      - uri
      - name
    type: object
  WorkspaceFoldersChangeEvent:
    additionalProperties: false
    description: The workspace folder change event.
    properties:
      added:
        description: The array of added workspace folders
        items:
          $ref: "#/definitions/WorkspaceFolder"
        type: array
      removed:
        description: The array of the removed workspace folders
        items:
          $ref: "#/definitions/WorkspaceFolder"
        type: array
    required:
      - added
      - removed
    type: object
  WorkspaceSymbolClientCapabilities:
    additionalProperties: false
    description: Client capabilities for a [WorkspaceSymbolRequest](#WorkspaceSymbolRequest).
    properties:
      dynamicRegistration:
        description: Symbol request supports dynamic registration.
        type: boolean
      symbolKind:
        additionalProperties: false
        description:
          Specific capabilities for the `SymbolKind` in the `workspace/symbol`
          request.
        properties:
          valueSet:
            description: "The symbol kind values the client supports. When this

              property exists the client also guarantees that it will

              handle values outside its set gracefully and falls back

              to a default value when unknown.


              If this property is not present the client only supports

              the symbol kinds from `File` to `Array` as defined in

              the initial version of the protocol."
            items:
              $ref: "#/definitions/SymbolKind"
            type: array
        type: object
    type: object
  WorkspaceSymbolOptions:
    additionalProperties: false
    description: Server capabilities for a [WorkspaceSymbolRequest](#WorkspaceSymbolRequest).
    properties:
      workDoneProgress:
        type: boolean
    type: object
  WorkspaceSymbolParams:
    additionalProperties: false
    description: The parameters of a [WorkspaceSymbolRequest](#WorkspaceSymbolRequest).
    properties:
      partialResultToken:
        $ref: "#/definitions/ProgressToken"
        description:
          "An optional token that a server can use to report partial results
          (e.g. streaming) to

          the client."
      query:
        description:
          "A query string to filter symbols by. Clients may send an empty

          string here to request all symbols."
        type: string
      workDoneToken:
        $ref: "#/definitions/ProgressToken"
        description: An optional token that a server can use to report work done progress.
    required:
      - query
    type: object
  _AnyFeature:
    anyOf:
      - $ref: "#/definitions/_CodeLensResolveFeature"
      - $ref: "#/definitions/_CompletionItemResolveFeature"
      - $ref: "#/definitions/_DocumentLinkResolveFeature"
      - $ref: "#/definitions/_InitializeFeature"
      - $ref: "#/definitions/_ShutdownFeature"
      - $ref: "#/definitions/_TextDocumentCodeActionFeature"
      - $ref: "#/definitions/_TextDocumentCodeLensFeature"
      - $ref: "#/definitions/_TextDocumentColorPresentationFeature"
      - $ref: "#/definitions/_TextDocumentCompletionFeature"
      - $ref: "#/definitions/_TextDocumentDeclarationFeature"
      - $ref: "#/definitions/_TextDocumentDefinitionFeature"
      - $ref: "#/definitions/_TextDocumentDocumentColorFeature"
      - $ref: "#/definitions/_TextDocumentDocumentHighlightFeature"
      - $ref: "#/definitions/_TextDocumentDocumentLinkFeature"
      - $ref: "#/definitions/_TextDocumentDocumentSymbolFeature"
      - $ref: "#/definitions/_TextDocumentFoldingRangeFeature"
      - $ref: "#/definitions/_TextDocumentFormattingFeature"
      - $ref: "#/definitions/_TextDocumentHoverFeature"
      - $ref: "#/definitions/_TextDocumentImplementationFeature"
      - $ref: "#/definitions/_TextDocumentOnTypeFormattingFeature"
      - $ref: "#/definitions/_TextDocumentPrepareRenameFeature"
      - $ref: "#/definitions/_TextDocumentRangeFormattingFeature"
      - $ref: "#/definitions/_TextDocumentReferencesFeature"
      - $ref: "#/definitions/_TextDocumentRenameFeature"
      - $ref: "#/definitions/_TextDocumentSignatureHelpFeature"
      - $ref: "#/definitions/_TextDocumentTypeDefinitionFeature"
      - $ref: "#/definitions/_TextDocumentWillSaveWaitUntilFeature"
      - $ref: "#/definitions/_WorkspaceExecuteCommandFeature"
      - $ref: "#/definitions/_WorkspaceSymbolFeature"
      - $ref: "#/definitions/_ClientRegisterCapabilityFeature"
      - $ref: "#/definitions/_ClientUnregisterCapabilityFeature"
      - $ref: "#/definitions/_WindowShowMessageRequestFeature"
      - $ref: "#/definitions/_WorkspaceApplyEditFeature"
      - $ref: "#/definitions/_WorkspaceConfigurationFeature"
      - $ref: "#/definitions/_WorkspaceWorkspaceFoldersFeature"
      - $ref: "#/definitions/_ExitFeature"
      - $ref: "#/definitions/_InitializedFeature"
      - $ref: "#/definitions/_TextDocumentDidChangeFeature"
      - $ref: "#/definitions/_TextDocumentDidCloseFeature"
      - $ref: "#/definitions/_TextDocumentDidOpenFeature"
      - $ref: "#/definitions/_TextDocumentDidSaveFeature"
      - $ref: "#/definitions/_TextDocumentWillSaveFeature"
      - $ref: "#/definitions/_WorkspaceDidChangeConfigurationFeature"
      - $ref: "#/definitions/_WorkspaceDidChangeWatchedFilesFeature"
      - $ref: "#/definitions/_WorkspaceDidChangeWorkspaceFoldersFeature"
      - $ref: "#/definitions/_CancelRequestFeature"
      - $ref: "#/definitions/_TelemetryEventFeature"
      - $ref: "#/definitions/_TextDocumentPublishDiagnosticsFeature"
      - $ref: "#/definitions/_WindowLogMessageFeature"
      - $ref: "#/definitions/_WindowShowMessageFeature"
  _CancelRequestFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_CancelRequestRequest"
    required:
      - request
    type: object
  _CancelRequestRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - $/cancelRequest
        type: string
      params:
        $ref: "#/definitions/CancelParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _ClientRegisterCapabilityFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_ClientRegisterCapabilityRequest"
      response:
        anyOf:
          - $ref: "#/definitions/_ClientRegisterCapabilityResponse"
          - $ref: "#/definitions/_ErrorResponse"
    required:
      - request
      - response
    type: object
  _ClientRegisterCapabilityRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - client/registerCapability
        type: string
      params:
        $ref: "#/definitions/RegistrationParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _ClientRegisterCapabilityResponse:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
          - "null"
      jsonrpc:
        type: string
      result:
        anyOf:
          - type: string
          - type: number
          - type: boolean
          - type: object
          - type: "null"
        description:
          "The result of a request. This member is REQUIRED on success.\r\
          \nThis member MUST NOT exist if there was an error invoking the method."
        type: "null"
    required:
      - id
      - jsonrpc
      - result
    type: object
  _ClientUnregisterCapabilityFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_ClientUnregisterCapabilityRequest"
      response:
        anyOf:
          - $ref: "#/definitions/_ClientUnregisterCapabilityResponse"
          - $ref: "#/definitions/_ErrorResponse"
    required:
      - request
      - response
    type: object
  _ClientUnregisterCapabilityRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - client/unregisterCapability
        type: string
      params:
        $ref: "#/definitions/UnregistrationParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _ClientUnregisterCapabilityResponse:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
          - "null"
      jsonrpc:
        type: string
      result:
        anyOf:
          - type: string
          - type: number
          - type: boolean
          - type: object
          - type: "null"
        description:
          "The result of a request. This member is REQUIRED on success.\r\
          \nThis member MUST NOT exist if there was an error invoking the method."
        type: "null"
    required:
      - id
      - jsonrpc
      - result
    type: object
  _CodeLensResolveFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_CodeLensResolveRequest"
      response:
        anyOf:
          - $ref: "#/definitions/_CodeLensResolveResponse"
          - $ref: "#/definitions/_ErrorResponse"
    required:
      - request
      - response
    type: object
  _CodeLensResolveRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - codeLens/resolve
        type: string
      params:
        $ref: "#/definitions/CodeLens"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _CodeLensResolveResponse:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
          - "null"
      jsonrpc:
        type: string
      result:
        anyOf:
          - type: string
          - type: number
          - type: boolean
          - type: object
          - type: "null"
        description:
          "The result of a request. This member is REQUIRED on success.\r\
          \nThis member MUST NOT exist if there was an error invoking the method."
    required:
      - id
      - jsonrpc
      - result
    type: object
  _CompletionItemResolveFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_CompletionItemResolveRequest"
      response:
        anyOf:
          - $ref: "#/definitions/_CompletionItemResolveResponse"
          - $ref: "#/definitions/_ErrorResponse"
    required:
      - request
      - response
    type: object
  _CompletionItemResolveRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - completionItem/resolve
        type: string
      params:
        $ref: "#/definitions/CompletionItem"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _CompletionItemResolveResponse:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
          - "null"
      jsonrpc:
        type: string
      result:
        anyOf:
          - type: string
          - type: number
          - type: boolean
          - type: object
          - type: "null"
        description:
          "The result of a request. This member is REQUIRED on success.\r\
          \nThis member MUST NOT exist if there was an error invoking the method."
    required:
      - id
      - jsonrpc
      - result
    type: object
  _DocumentLinkResolveFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_DocumentLinkResolveRequest"
      response:
        anyOf:
          - $ref: "#/definitions/_DocumentLinkResolveResponse"
          - $ref: "#/definitions/_ErrorResponse"
    required:
      - request
      - response
    type: object
  _DocumentLinkResolveRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - documentLink/resolve
        type: string
      params:
        $ref: "#/definitions/DocumentLink"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _DocumentLinkResolveResponse:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
          - "null"
      jsonrpc:
        type: string
      result:
        anyOf:
          - type: string
          - type: number
          - type: boolean
          - type: object
          - type: "null"
        description:
          "The result of a request. This member is REQUIRED on success.\r\
          \nThis member MUST NOT exist if there was an error invoking the method."
    required:
      - id
      - jsonrpc
      - result
    type: object
  _ErrorResponse:
    additionalProperties: false
    properties:
      error:
        $ref: "#/definitions/ResponseErrorLiteral<any>"
        description: The error object in case a request fails.
      id:
        description: The request id.
        type:
          - number
          - string
          - "null"
      jsonrpc:
        type: string
    required:
      - error
      - id
      - jsonrpc
    type: object
  _ExitFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_ExitRequest"
    required:
      - request
    type: object
  _ExitRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - exit
        type: string
      params:
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
    type: object
  _InitializeFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_InitializeRequest"
      response:
        anyOf:
          - $ref: "#/definitions/_InitializeResponse"
          - $ref: "#/definitions/_ErrorResponse"
    required:
      - request
      - response
    type: object
  _InitializeRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - initialize
        type: string
      params:
        $ref: "#/definitions/InitializeParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _InitializeResponse:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
          - "null"
      jsonrpc:
        type: string
      result:
        anyOf:
          - type: string
          - type: number
          - type: boolean
          - type: object
          - type: "null"
        description:
          "The result of a request. This member is REQUIRED on success.\r\
          \nThis member MUST NOT exist if there was an error invoking the method."
    required:
      - id
      - jsonrpc
      - result
    type: object
  _InitializedFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_InitializedRequest"
    required:
      - request
    type: object
  _InitializedRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - initialized
        type: string
      params:
        $ref: "#/definitions/InitializedParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _ShutdownFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_ShutdownRequest"
      response:
        anyOf:
          - $ref: "#/definitions/_ShutdownResponse"
          - $ref: "#/definitions/_ErrorResponse"
    required:
      - request
      - response
    type: object
  _ShutdownRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - shutdown
        type: string
      params:
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
    type: object
  _ShutdownResponse:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
          - "null"
      jsonrpc:
        type: string
      result:
        anyOf:
          - type: string
          - type: number
          - type: boolean
          - type: object
          - type: "null"
        description:
          "The result of a request. This member is REQUIRED on success.\r\
          \nThis member MUST NOT exist if there was an error invoking the method."
        type: "null"
    required:
      - id
      - jsonrpc
      - result
    type: object
  _TelemetryEventFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_TelemetryEventRequest"
    required:
      - request
    type: object
  _TelemetryEventRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - telemetry/event
        type: string
      params:
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
    type: object
  _TextDocumentCodeActionFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_TextDocumentCodeActionRequest"
      response:
        anyOf:
          - $ref: "#/definitions/_TextDocumentCodeActionResponse"
          - $ref: "#/definitions/_ErrorResponse"
    required:
      - request
      - response
    type: object
  _TextDocumentCodeActionRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - textDocument/codeAction
        type: string
      params:
        $ref: "#/definitions/CodeActionParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _TextDocumentCodeActionResponse:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
          - "null"
      jsonrpc:
        type: string
      result:
        anyOf:
          - items:
              anyOf:
                - $ref: "#/definitions/Command"
                - $ref: "#/definitions/CodeAction"
            type: array
          - type: "null"
        description:
          "The result of a request. This member is REQUIRED on success.\r\
          \nThis member MUST NOT exist if there was an error invoking the method."
    required:
      - id
      - jsonrpc
      - result
    type: object
  _TextDocumentCodeLensFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_TextDocumentCodeLensRequest"
      response:
        anyOf:
          - $ref: "#/definitions/_TextDocumentCodeLensResponse"
          - $ref: "#/definitions/_ErrorResponse"
    required:
      - request
      - response
    type: object
  _TextDocumentCodeLensRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - textDocument/codeLens
        type: string
      params:
        $ref: "#/definitions/CodeLensParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _TextDocumentCodeLensResponse:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
          - "null"
      jsonrpc:
        type: string
      result:
        anyOf:
          - items:
              $ref: "#/definitions/CodeLens"
            type: array
          - type: "null"
        description:
          "The result of a request. This member is REQUIRED on success.\r\
          \nThis member MUST NOT exist if there was an error invoking the method."
    required:
      - id
      - jsonrpc
      - result
    type: object
  _TextDocumentColorPresentationFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_TextDocumentColorPresentationRequest"
      response:
        anyOf:
          - $ref: "#/definitions/_TextDocumentColorPresentationResponse"
          - $ref: "#/definitions/_ErrorResponse"
    required:
      - request
      - response
    type: object
  _TextDocumentColorPresentationRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - textDocument/colorPresentation
        type: string
      params:
        $ref: "#/definitions/ColorPresentationParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _TextDocumentColorPresentationResponse:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
          - "null"
      jsonrpc:
        type: string
      result:
        anyOf:
          - type: string
          - type: number
          - type: boolean
          - type: object
          - type: "null"
        description:
          "The result of a request. This member is REQUIRED on success.\r\
          \nThis member MUST NOT exist if there was an error invoking the method."
        items:
          $ref: "#/definitions/ColorPresentation"
        type: array
    required:
      - id
      - jsonrpc
      - result
    type: object
  _TextDocumentCompletionFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_TextDocumentCompletionRequest"
      response:
        anyOf:
          - $ref: "#/definitions/_TextDocumentCompletionResponse"
          - $ref: "#/definitions/_ErrorResponse"
    required:
      - request
      - response
    type: object
  _TextDocumentCompletionRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - textDocument/completion
        type: string
      params:
        $ref: "#/definitions/CompletionParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _TextDocumentCompletionResponse:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
          - "null"
      jsonrpc:
        type: string
      result:
        anyOf:
          - items:
              $ref: "#/definitions/CompletionItem"
            type: array
          - $ref: "#/definitions/CompletionList"
          - type: "null"
        description:
          "The result of a request. This member is REQUIRED on success.\r\
          \nThis member MUST NOT exist if there was an error invoking the method."
    required:
      - id
      - jsonrpc
      - result
    type: object
  _TextDocumentDeclarationFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_TextDocumentDeclarationRequest"
      response:
        anyOf:
          - $ref: "#/definitions/_TextDocumentDeclarationResponse"
          - $ref: "#/definitions/_ErrorResponse"
    required:
      - request
      - response
    type: object
  _TextDocumentDeclarationRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - textDocument/declaration
        type: string
      params:
        $ref: "#/definitions/TextDocumentPositionParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _TextDocumentDeclarationResponse:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
          - "null"
      jsonrpc:
        type: string
      result:
        anyOf:
          - items:
              $ref: "#/definitions/Location"
            type: array
          - type: "null"
        description:
          "The result of a request. This member is REQUIRED on success.\r\
          \nThis member MUST NOT exist if there was an error invoking the method."
    required:
      - id
      - jsonrpc
      - result
    type: object
  _TextDocumentDefinitionFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_TextDocumentDefinitionRequest"
      response:
        anyOf:
          - $ref: "#/definitions/_TextDocumentDefinitionResponse"
          - $ref: "#/definitions/_ErrorResponse"
    required:
      - request
      - response
    type: object
  _TextDocumentDefinitionRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - textDocument/definition
        type: string
      params:
        $ref: "#/definitions/TextDocumentPositionParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _TextDocumentDefinitionResponse:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
          - "null"
      jsonrpc:
        type: string
      result:
        anyOf:
          - items:
              $ref: "#/definitions/Location"
            type: array
          - type: "null"
        description:
          "The result of a request. This member is REQUIRED on success.\r\
          \nThis member MUST NOT exist if there was an error invoking the method."
    required:
      - id
      - jsonrpc
      - result
    type: object
  _TextDocumentDidChangeFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_TextDocumentDidChangeRequest"
    required:
      - request
    type: object
  _TextDocumentDidChangeRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - textDocument/didChange
        type: string
      params:
        $ref: "#/definitions/DidChangeTextDocumentParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _TextDocumentDidCloseFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_TextDocumentDidCloseRequest"
    required:
      - request
    type: object
  _TextDocumentDidCloseRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - textDocument/didClose
        type: string
      params:
        $ref: "#/definitions/DidCloseTextDocumentParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _TextDocumentDidOpenFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_TextDocumentDidOpenRequest"
    required:
      - request
    type: object
  _TextDocumentDidOpenRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - textDocument/didOpen
        type: string
      params:
        $ref: "#/definitions/DidOpenTextDocumentParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _TextDocumentDidSaveFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_TextDocumentDidSaveRequest"
    required:
      - request
    type: object
  _TextDocumentDidSaveRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - textDocument/didSave
        type: string
      params:
        $ref: "#/definitions/DidSaveTextDocumentParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _TextDocumentDocumentColorFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_TextDocumentDocumentColorRequest"
      response:
        anyOf:
          - $ref: "#/definitions/_TextDocumentDocumentColorResponse"
          - $ref: "#/definitions/_ErrorResponse"
    required:
      - request
      - response
    type: object
  _TextDocumentDocumentColorRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - textDocument/documentColor
        type: string
      params:
        $ref: "#/definitions/DocumentColorParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _TextDocumentDocumentColorResponse:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
          - "null"
      jsonrpc:
        type: string
      result:
        anyOf:
          - type: string
          - type: number
          - type: boolean
          - type: object
          - type: "null"
        description:
          "The result of a request. This member is REQUIRED on success.\r\
          \nThis member MUST NOT exist if there was an error invoking the method."
        items:
          $ref: "#/definitions/ColorInformation"
        type: array
    required:
      - id
      - jsonrpc
      - result
    type: object
  _TextDocumentDocumentHighlightFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_TextDocumentDocumentHighlightRequest"
      response:
        anyOf:
          - $ref: "#/definitions/_TextDocumentDocumentHighlightResponse"
          - $ref: "#/definitions/_ErrorResponse"
    required:
      - request
      - response
    type: object
  _TextDocumentDocumentHighlightRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - textDocument/documentHighlight
        type: string
      params:
        $ref: "#/definitions/TextDocumentPositionParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _TextDocumentDocumentHighlightResponse:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
          - "null"
      jsonrpc:
        type: string
      result:
        anyOf:
          - items:
              $ref: "#/definitions/DocumentHighlight"
            type: array
          - type: "null"
        description:
          "The result of a request. This member is REQUIRED on success.\r\
          \nThis member MUST NOT exist if there was an error invoking the method."
    required:
      - id
      - jsonrpc
      - result
    type: object
  _TextDocumentDocumentLinkFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_TextDocumentDocumentLinkRequest"
      response:
        anyOf:
          - $ref: "#/definitions/_TextDocumentDocumentLinkResponse"
          - $ref: "#/definitions/_ErrorResponse"
    required:
      - request
      - response
    type: object
  _TextDocumentDocumentLinkRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - textDocument/documentLink
        type: string
      params:
        $ref: "#/definitions/DocumentLinkParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _TextDocumentDocumentLinkResponse:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
          - "null"
      jsonrpc:
        type: string
      result:
        anyOf:
          - items:
              $ref: "#/definitions/DocumentLink"
            type: array
          - type: "null"
        description:
          "The result of a request. This member is REQUIRED on success.\r\
          \nThis member MUST NOT exist if there was an error invoking the method."
    required:
      - id
      - jsonrpc
      - result
    type: object
  _TextDocumentDocumentSymbolFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_TextDocumentDocumentSymbolRequest"
      response:
        anyOf:
          - $ref: "#/definitions/_TextDocumentDocumentSymbolResponse"
          - $ref: "#/definitions/_ErrorResponse"
    required:
      - request
      - response
    type: object
  _TextDocumentDocumentSymbolRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - textDocument/documentSymbol
        type: string
      params:
        $ref: "#/definitions/DocumentSymbolParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _TextDocumentDocumentSymbolResponse:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
          - "null"
      jsonrpc:
        type: string
      result:
        anyOf:
          - items:
              $ref: "#/definitions/DocumentSymbol"
            type: array
          - items:
              $ref: "#/definitions/SymbolInformation"
            type: array
          - type: "null"
        description:
          "The result of a request. This member is REQUIRED on success.\r\
          \nThis member MUST NOT exist if there was an error invoking the method."
    required:
      - id
      - jsonrpc
      - result
    type: object
  _TextDocumentFoldingRangeFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_TextDocumentFoldingRangeRequest"
      response:
        anyOf:
          - $ref: "#/definitions/_TextDocumentFoldingRangeResponse"
          - $ref: "#/definitions/_ErrorResponse"
    required:
      - request
      - response
    type: object
  _TextDocumentFoldingRangeRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - textDocument/foldingRange
        type: string
      params:
        $ref: "#/definitions/FoldingRangeParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _TextDocumentFoldingRangeResponse:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
          - "null"
      jsonrpc:
        type: string
      result:
        anyOf:
          - items:
              $ref: "#/definitions/FoldingRange"
            type: array
          - type: "null"
        description:
          "The result of a request. This member is REQUIRED on success.\r\
          \nThis member MUST NOT exist if there was an error invoking the method."
    required:
      - id
      - jsonrpc
      - result
    type: object
  _TextDocumentFormattingFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_TextDocumentFormattingRequest"
      response:
        anyOf:
          - $ref: "#/definitions/_TextDocumentFormattingResponse"
          - $ref: "#/definitions/_ErrorResponse"
    required:
      - request
      - response
    type: object
  _TextDocumentFormattingRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - textDocument/formatting
        type: string
      params:
        $ref: "#/definitions/DocumentFormattingParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _TextDocumentFormattingResponse:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
          - "null"
      jsonrpc:
        type: string
      result:
        anyOf:
          - items:
              $ref: "#/definitions/TextEdit"
            type: array
          - type: "null"
        description:
          "The result of a request. This member is REQUIRED on success.\r\
          \nThis member MUST NOT exist if there was an error invoking the method."
    required:
      - id
      - jsonrpc
      - result
    type: object
  _TextDocumentHoverFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_TextDocumentHoverRequest"
      response:
        anyOf:
          - $ref: "#/definitions/_TextDocumentHoverResponse"
          - $ref: "#/definitions/_ErrorResponse"
    required:
      - request
      - response
    type: object
  _TextDocumentHoverRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - textDocument/hover
        type: string
      params:
        $ref: "#/definitions/TextDocumentPositionParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _TextDocumentHoverResponse:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
          - "null"
      jsonrpc:
        type: string
      result:
        anyOf:
          - $ref: "#/definitions/Hover"
          - type: "null"
        description:
          "The result of a request. This member is REQUIRED on success.\r\
          \nThis member MUST NOT exist if there was an error invoking the method."
    required:
      - id
      - jsonrpc
      - result
    type: object
  _TextDocumentImplementationFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_TextDocumentImplementationRequest"
      response:
        anyOf:
          - $ref: "#/definitions/_TextDocumentImplementationResponse"
          - $ref: "#/definitions/_ErrorResponse"
    required:
      - request
      - response
    type: object
  _TextDocumentImplementationRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - textDocument/implementation
        type: string
      params:
        $ref: "#/definitions/TextDocumentPositionParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _TextDocumentImplementationResponse:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
          - "null"
      jsonrpc:
        type: string
      result:
        anyOf:
          - items:
              $ref: "#/definitions/Location"
            type: array
          - type: "null"
        description:
          "The result of a request. This member is REQUIRED on success.\r\
          \nThis member MUST NOT exist if there was an error invoking the method."
    required:
      - id
      - jsonrpc
      - result
    type: object
  _TextDocumentOnTypeFormattingFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_TextDocumentOnTypeFormattingRequest"
      response:
        anyOf:
          - $ref: "#/definitions/_TextDocumentOnTypeFormattingResponse"
          - $ref: "#/definitions/_ErrorResponse"
    required:
      - request
      - response
    type: object
  _TextDocumentOnTypeFormattingRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - textDocument/onTypeFormatting
        type: string
      params:
        $ref: "#/definitions/DocumentOnTypeFormattingParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _TextDocumentOnTypeFormattingResponse:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
          - "null"
      jsonrpc:
        type: string
      result:
        anyOf:
          - items:
              $ref: "#/definitions/TextEdit"
            type: array
          - type: "null"
        description:
          "The result of a request. This member is REQUIRED on success.\r\
          \nThis member MUST NOT exist if there was an error invoking the method."
    required:
      - id
      - jsonrpc
      - result
    type: object
  _TextDocumentPrepareRenameFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_TextDocumentPrepareRenameRequest"
      response:
        anyOf:
          - $ref: "#/definitions/_TextDocumentPrepareRenameResponse"
          - $ref: "#/definitions/_ErrorResponse"
    required:
      - request
      - response
    type: object
  _TextDocumentPrepareRenameRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - textDocument/prepareRename
        type: string
      params:
        $ref: "#/definitions/TextDocumentPositionParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _TextDocumentPrepareRenameResponse:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
          - "null"
      jsonrpc:
        type: string
      result:
        anyOf:
          - $ref: "#/definitions/Range"
          - additionalProperties: false
            properties:
              placeholder:
                type: string
              range:
                $ref: "#/definitions/Range"
            required:
              - range
              - placeholder
            type: object
          - type: "null"
        description:
          "The result of a request. This member is REQUIRED on success.\r\
          \nThis member MUST NOT exist if there was an error invoking the method."
    required:
      - id
      - jsonrpc
      - result
    type: object
  _TextDocumentPublishDiagnosticsFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_TextDocumentPublishDiagnosticsRequest"
    required:
      - request
    type: object
  _TextDocumentPublishDiagnosticsRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - textDocument/publishDiagnostics
        type: string
      params:
        $ref: "#/definitions/PublishDiagnosticsParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _TextDocumentRangeFormattingFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_TextDocumentRangeFormattingRequest"
      response:
        anyOf:
          - $ref: "#/definitions/_TextDocumentRangeFormattingResponse"
          - $ref: "#/definitions/_ErrorResponse"
    required:
      - request
      - response
    type: object
  _TextDocumentRangeFormattingRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - textDocument/rangeFormatting
        type: string
      params:
        $ref: "#/definitions/DocumentRangeFormattingParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _TextDocumentRangeFormattingResponse:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
          - "null"
      jsonrpc:
        type: string
      result:
        anyOf:
          - items:
              $ref: "#/definitions/TextEdit"
            type: array
          - type: "null"
        description:
          "The result of a request. This member is REQUIRED on success.\r\
          \nThis member MUST NOT exist if there was an error invoking the method."
    required:
      - id
      - jsonrpc
      - result
    type: object
  _TextDocumentReferencesFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_TextDocumentReferencesRequest"
      response:
        anyOf:
          - $ref: "#/definitions/_TextDocumentReferencesResponse"
          - $ref: "#/definitions/_ErrorResponse"
    required:
      - request
      - response
    type: object
  _TextDocumentReferencesRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - textDocument/references
        type: string
      params:
        $ref: "#/definitions/ReferenceParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _TextDocumentReferencesResponse:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
          - "null"
      jsonrpc:
        type: string
      result:
        anyOf:
          - items:
              $ref: "#/definitions/Location"
            type: array
          - type: "null"
        description:
          "The result of a request. This member is REQUIRED on success.\r\
          \nThis member MUST NOT exist if there was an error invoking the method."
    required:
      - id
      - jsonrpc
      - result
    type: object
  _TextDocumentRenameFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_TextDocumentRenameRequest"
      response:
        anyOf:
          - $ref: "#/definitions/_TextDocumentRenameResponse"
          - $ref: "#/definitions/_ErrorResponse"
    required:
      - request
      - response
    type: object
  _TextDocumentRenameRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - textDocument/rename
        type: string
      params:
        $ref: "#/definitions/RenameParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _TextDocumentRenameResponse:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
          - "null"
      jsonrpc:
        type: string
      result:
        anyOf:
          - $ref: "#/definitions/WorkspaceEdit"
          - type: "null"
        description:
          "The result of a request. This member is REQUIRED on success.\r\
          \nThis member MUST NOT exist if there was an error invoking the method."
    required:
      - id
      - jsonrpc
      - result
    type: object
  _TextDocumentSignatureHelpFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_TextDocumentSignatureHelpRequest"
      response:
        anyOf:
          - $ref: "#/definitions/_TextDocumentSignatureHelpResponse"
          - $ref: "#/definitions/_ErrorResponse"
    required:
      - request
      - response
    type: object
  _TextDocumentSignatureHelpRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - textDocument/signatureHelp
        type: string
      params:
        $ref: "#/definitions/TextDocumentPositionParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _TextDocumentSignatureHelpResponse:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
          - "null"
      jsonrpc:
        type: string
      result:
        anyOf:
          - $ref: "#/definitions/SignatureHelp"
          - type: "null"
        description:
          "The result of a request. This member is REQUIRED on success.\r\
          \nThis member MUST NOT exist if there was an error invoking the method."
    required:
      - id
      - jsonrpc
      - result
    type: object
  _TextDocumentTypeDefinitionFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_TextDocumentTypeDefinitionRequest"
      response:
        anyOf:
          - $ref: "#/definitions/_TextDocumentTypeDefinitionResponse"
          - $ref: "#/definitions/_ErrorResponse"
    required:
      - request
      - response
    type: object
  _TextDocumentTypeDefinitionRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - textDocument/typeDefinition
        type: string
      params:
        $ref: "#/definitions/TextDocumentPositionParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _TextDocumentTypeDefinitionResponse:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
          - "null"
      jsonrpc:
        type: string
      result:
        anyOf:
          - items:
              $ref: "#/definitions/Location"
            type: array
          - type: "null"
        description:
          "The result of a request. This member is REQUIRED on success.\r\
          \nThis member MUST NOT exist if there was an error invoking the method."
    required:
      - id
      - jsonrpc
      - result
    type: object
  _TextDocumentWillSaveFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_TextDocumentWillSaveRequest"
    required:
      - request
    type: object
  _TextDocumentWillSaveRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - textDocument/willSave
        type: string
      params:
        $ref: "#/definitions/WillSaveTextDocumentParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _TextDocumentWillSaveWaitUntilFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_TextDocumentWillSaveWaitUntilRequest"
    required:
      - request
    type: object
  _TextDocumentWillSaveWaitUntilRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - textDocument/willSaveWaitUntil
        type: string
      params:
        $ref: "#/definitions/WillSaveTextDocumentParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _WindowLogMessageFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_WindowLogMessageRequest"
    required:
      - request
    type: object
  _WindowLogMessageRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - window/logMessage
        type: string
      params:
        $ref: "#/definitions/LogMessageParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _WindowShowMessageFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_WindowShowMessageRequest"
    required:
      - request
    type: object
  _WindowShowMessageRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - window/showMessage
        type: string
      params:
        $ref: "#/definitions/ShowMessageParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _WindowShowMessageRequestFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_WindowShowMessageRequestRequest"
      response:
        anyOf:
          - $ref: "#/definitions/_WindowShowMessageRequestResponse"
          - $ref: "#/definitions/_ErrorResponse"
    required:
      - request
      - response
    type: object
  _WindowShowMessageRequestRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - window/showMessageRequest
        type: string
      params:
        $ref: "#/definitions/ShowMessageRequestParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _WindowShowMessageRequestResponse:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
          - "null"
      jsonrpc:
        type: string
      result:
        anyOf:
          - $ref: "#/definitions/MessageActionItem"
          - type: "null"
        description:
          "The result of a request. This member is REQUIRED on success.\r\
          \nThis member MUST NOT exist if there was an error invoking the method."
    required:
      - id
      - jsonrpc
      - result
    type: object
  _WorkspaceApplyEditFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_WorkspaceApplyEditRequest"
      response:
        anyOf:
          - $ref: "#/definitions/_WorkspaceApplyEditResponse"
          - $ref: "#/definitions/_ErrorResponse"
    required:
      - request
      - response
    type: object
  _WorkspaceApplyEditRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - workspace/applyEdit
        type: string
      params:
        $ref: "#/definitions/ApplyWorkspaceEditParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _WorkspaceApplyEditResponse:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
          - "null"
      jsonrpc:
        type: string
      result:
        anyOf:
          - type: string
          - type: number
          - type: boolean
          - type: object
          - type: "null"
        description:
          "The result of a request. This member is REQUIRED on success.\r\
          \nThis member MUST NOT exist if there was an error invoking the method."
    required:
      - id
      - jsonrpc
      - result
    type: object
  _WorkspaceConfigurationFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_WorkspaceConfigurationRequest"
      response:
        anyOf:
          - $ref: "#/definitions/_WorkspaceConfigurationResponse"
          - $ref: "#/definitions/_ErrorResponse"
    required:
      - request
      - response
    type: object
  _WorkspaceConfigurationRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - workspace/configuration
        type: string
      params:
        $ref: "#/definitions/ConfigurationParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _WorkspaceConfigurationResponse:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
          - "null"
      jsonrpc:
        type: string
      result:
        anyOf:
          - type: string
          - type: number
          - type: boolean
          - type: object
          - type: "null"
        description:
          "The result of a request. This member is REQUIRED on success.\r\
          \nThis member MUST NOT exist if there was an error invoking the method."
        items: {}
        type: array
    required:
      - id
      - jsonrpc
      - result
    type: object
  _WorkspaceDidChangeConfigurationFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_WorkspaceDidChangeConfigurationRequest"
    required:
      - request
    type: object
  _WorkspaceDidChangeConfigurationRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - workspace/didChangeConfiguration
        type: string
      params:
        $ref: "#/definitions/DidChangeConfigurationParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _WorkspaceDidChangeWatchedFilesFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_WorkspaceDidChangeWatchedFilesRequest"
    required:
      - request
    type: object
  _WorkspaceDidChangeWatchedFilesRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - workspace/didChangeWatchedFiles
        type: string
      params:
        $ref: "#/definitions/DidChangeWatchedFilesParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _WorkspaceDidChangeWorkspaceFoldersFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_WorkspaceDidChangeWorkspaceFoldersRequest"
    required:
      - request
    type: object
  _WorkspaceDidChangeWorkspaceFoldersRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - workspace/didChangeWorkspaceFolders
        type: string
      params:
        $ref: "#/definitions/DidChangeWorkspaceFoldersParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _WorkspaceExecuteCommandFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_WorkspaceExecuteCommandRequest"
      response:
        anyOf:
          - $ref: "#/definitions/_WorkspaceExecuteCommandResponse"
          - $ref: "#/definitions/_ErrorResponse"
    required:
      - request
      - response
    type: object
  _WorkspaceExecuteCommandRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - workspace/executeCommand
        type: string
      params:
        $ref: "#/definitions/ExecuteCommandParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _WorkspaceExecuteCommandResponse:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
          - "null"
      jsonrpc:
        type: string
      result:
        anyOf:
          - {}
          - type: "null"
        description:
          "The result of a request. This member is REQUIRED on success.\r\
          \nThis member MUST NOT exist if there was an error invoking the method."
    required:
      - id
      - jsonrpc
      - result
    type: object
  _WorkspaceSymbolFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_WorkspaceSymbolRequest"
      response:
        anyOf:
          - $ref: "#/definitions/_WorkspaceSymbolResponse"
          - $ref: "#/definitions/_ErrorResponse"
    required:
      - request
      - response
    type: object
  _WorkspaceSymbolRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - workspace/symbol
        type: string
      params:
        $ref: "#/definitions/WorkspaceSymbolParams"
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
      - params
    type: object
  _WorkspaceSymbolResponse:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
          - "null"
      jsonrpc:
        type: string
      result:
        anyOf:
          - items:
              $ref: "#/definitions/SymbolInformation"
            type: array
          - type: "null"
        description:
          "The result of a request. This member is REQUIRED on success.\r\
          \nThis member MUST NOT exist if there was an error invoking the method."
    required:
      - id
      - jsonrpc
      - result
    type: object
  _WorkspaceWorkspaceFoldersFeature:
    additionalProperties: false
    properties:
      request:
        $ref: "#/definitions/_WorkspaceWorkspaceFoldersRequest"
      response:
        anyOf:
          - $ref: "#/definitions/_WorkspaceWorkspaceFoldersResponse"
          - $ref: "#/definitions/_ErrorResponse"
    required:
      - request
      - response
    type: object
  _WorkspaceWorkspaceFoldersRequest:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
      jsonrpc:
        type: string
      method:
        description: The method to be invoked.
        enum:
          - workspace/workspaceFolders
        type: string
      params:
        description: The method's params.
    required:
      - id
      - jsonrpc
      - method
    type: object
  _WorkspaceWorkspaceFoldersResponse:
    additionalProperties: false
    properties:
      id:
        description: The request id.
        type:
          - number
          - string
          - "null"
      jsonrpc:
        type: string
      result:
        anyOf:
          - items:
              $ref: "#/definitions/WorkspaceFolder"
            type: array
          - type: "null"
        description:
          "The result of a request. This member is REQUIRED on success.\r\
          \nThis member MUST NOT exist if there was an error invoking the method."
    required:
      - id
      - jsonrpc
      - result
    type: object

```